## **Installation and Import Stuff**

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd gdrive/My Drive/gaguilar

/content/gdrive/.shortcut-targets-by-id/1D5I4ZlzhFow8fFGxbipMXBZSuRXJ01yA/gaguilar


In [ ]:
import tensorflow as tf
print(tf.__version__)

seed_number = 42

print("GPUs Available: ", tf.config.experimental.list_physical_devices('GPU'))
physical_devices= tf.config.experimental.list_physical_devices('GPU')
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

tf.config.experimental.set_memory_growth(physical_devices[0], True)

# tf.compat.v1.set_random_seed(seed_value)

2.5.0
GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13413609558998971057
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 16183459840
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6995042542037162829
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


In [ ]:
import keras
keras.__version__

'2.5.0'

In [ ]:
!pip3 install emoji
!pip3 install python-crfsuite
!pip3 install sklearn-crfsuite

In [ ]:
# print('intermittent')
import os
# import tensorflow as tf
# # os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
# tf.set_random_seed(42)

os.environ['PYTHONHASHSEED']=str(seed_number)

In [ ]:
from keras import backend as K
config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,inter_op_parallelism_threads=1)
tf.compat.v1.set_random_seed(seed_number)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=config)
K.set_session(sess)

In [ ]:
!pip3 install emoji

In [ ]:
!pip3 install datasets

In [ ]:
nltk.download('stopwords')
nltk.download('gutenberg')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [ ]:
import pandas as pd
import numpy as np
import re
import string

from datasets import load_dataset, load_metric, ClassLabel, Sequence
import random
import torch
import copy

# import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split

import math
import shutil

np.random.seed(seed_number)

torch.autograd.set_detect_anomaly(True)

## **Setting some global stuff**

In [ ]:
import torch
# Set device
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(device)

# seed = 123
torch.manual_seed(seed_number)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed_number)

cuda


In [ ]:
# import pandas as pd
# import numpy as np

learning_rate = 0.0001

class PhraseEmbedding(nn.Module):

    def __init__(self,input_size,output_size,device):
        super(PhraseEmbedding, self).__init__()
        self.print_once=True
        self.dense_layer = nn.Linear(input_size,output_size)
        self.non_linear_layer = nn.Tanh()
        self.cosine_layer = nn.CosineSimilarity(dim=0)
        self.device = device
        return

    def encode(self, input_embedding):

        # print(input_embedding.size())
        input_sentence_embedding = input_embedding.squeeze(0)
        # print(input_sentence_embedding.size())
        # print('-----')

        # Max Pool
        # max_pooled_embedding = torch.max(input_sentence_embedding,dim=0)

        #Average Pool
        average_pooled_embedding = torch.mean(input_sentence_embedding,dim=0).to(device=self.device)
        # print(average_pooled_embedding.size())

        x = self.dense_layer(average_pooled_embedding)
        # print(x.size())

        out = self.non_linear_layer(x)
        # print(out.size())
        return out

    def forward(self, input_tuple):
        # print(len(input_tuple))
        input_source = input_tuple[0]
        input_target = input_tuple[1]

        output_source = self.encode(input_source)
        output_target = self.encode(input_target)

        similarity = self.cosine_layer(output_source, output_target)
        # print(similarity)
        return similarity

    def getEmbedding(self, input_embeddings):
        with torch.no_grad():
            return self.encode(input_embeddings)

## **Initialization of some components**

In [ ]:
from nltk.corpus import stopwords
from nltk.corpus import gutenberg
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer

gutenberg_text = ""
for file_id in gutenberg.fileids():
    gutenberg_text += gutenberg.raw(file_id)
tokenizer_trainer = PunktTrainer()
tokenizer_trainer.INCLUDE_ALL_COLLOCS = True
tokenizer_trainer.train(gutenberg_text)

sentence_tokenizer = PunktSentenceTokenizer(tokenizer_trainer.get_params())
sentence_tokenizer._params.abbrev_types.add('dr')
sentence_tokenizer._params.abbrev_types.add('c.j')
sentence_tokenizer._params.abbrev_types.add('u.s')
sentence_tokenizer._params.abbrev_types.add('u.s.a')
sentence_tokenizer._params.abbrev_types.add('ret.')
sentence_tokenizer._params.abbrev_types.add('rep.')

In [ ]:
from os import path

# from entityEmbedding import phraseEmbedding
def load_ckp(checkpoint_fpath, model, optimizer):
    checkpoint = torch.load(checkpoint_fpath)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    return model, optimizer, checkpoint['epoch']

# Initialize network
output_embedding_size = 100

phraseEmbeddingModel = PhraseEmbedding(100, output_embedding_size, device).to(device)

#Loss and Optimizer
criterion = nn.MSELoss(reduction='mean' )
optimizer = optim.Adam(phraseEmbeddingModel.parameters(), lr=learning_rate, weight_decay=0.0001)

# define checkpoint saved path
ckp_path = "entityEmbedding/model_checkpoints/checkpoint_model100.pt" #100

if(path.exists(ckp_path)):
    # load the saved checkpoint
    entityPhraseEmbedder, optimizer, start_epoch = load_ckp(ckp_path, phraseEmbeddingModel, optimizer)

    print("starting with model at epoch:", start_epoch)

starting with model at epoch: 20


In [ ]:
import numpy as np
np.random.seed(seed_number)

## **Entity Classifier**

In [ ]:
import pandas as pd
import time

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split

# import numpy as np
# np.random.seed(seed_number)



# 2 output_classes: 'entity'/'non-entity'; so sigmoid transformation would suffice

class NN(nn.Module):
  def __init__(self,input_size):
    super(NN, self).__init__()
    self.linear1 = nn.Linear(input_size,50)
    self.linear2 = nn.Linear(50,25)
    self.linear3 = nn.Linear(25,1)
    # self.linear4 = nn.Linear(6,1)
    self.sigmoid_layer = nn.Sigmoid()
      
  def forward(self, x): 
    x = F.relu(self.linear1(x))
    x = F.relu(self.linear2(x))
    # x = F.relu(self.linear3(x))
    # x = F.relu(self.linear4(x))
    x = self.linear3(x)
    out = self.sigmoid_layer(x)
    return out

class EntityClassifier():

    def __init__(self,training_file, to_train, device):

        # # using embedding + syntax features
        # self.combined_feature_list=['length']+['cap','substring-cap','s-o-sCap','all-cap','non-cap','non-discriminative']+['cf_'+str(i) for i in range(100)]

        # separately using only semantic features
        self.combined_feature_list=['length']+['cf_'+str(i) for i in range(100)]

        self.relevant_columns = ['normalized_length'#,
            # 'normalized_cap',
            # 'normalized_substring-cap',
            # 'normalized_s-o-sCap',
            # 'normalized_all-cap',
            # 'normalized_non-cap',
            # 'normalized_non-discriminative'
            ]+['normalized_cf_'+str(i) for i in range(100)]

        # create scaler
        self.scaler = StandardScaler()
        # self.scaler = MinMaxScaler()
        
        #initialize the classifier model
        self.classifier = NN(len(self.relevant_columns)).to(device)
        #Loss and Optimizer
        self.ec_criterion = nn.BCELoss(reduction='mean' )
        self.ec_optimizer = optim.Adam(self.classifier.parameters(), lr = 0.0001, weight_decay=0.0001)
        self.ec_batch_size = 32
        self.ec_num_epochs = 1000
        self.patience = 20


        if(to_train):

            self.train = pd.read_csv(training_file,delimiter=",",sep='\s*,\s*')
            #pre-processing : this completes the global average pooling
            
            max_length=self.train['length'].max()
            self.train['normalized_length']= self.train['length']/max_length
            for column in self.combined_feature_list[1:]:
                self.train['normalized_'+column]=self.train[column]/self.train['cumulative']
            
            #Loading the data
            training_inputs_array = self.train[self.relevant_columns].to_numpy()
            training_targets_array = self.train['class'].astype(float).to_numpy()

            # # fit and transform in one step
            training_inputs_array_standardized = self.scaler.fit_transform(training_inputs_array)

            training_inputs = torch.from_numpy(training_inputs_array_standardized).type(torch.float)
            # training_inputs = torch.from_numpy(training_inputs_array).type(torch.float)
            training_targets = torch.from_numpy(training_targets_array).type(torch.float)

            print('Input Shape: ', training_inputs.shape)
            print('Output Shape: ', training_targets.shape)

            dataset = TensorDataset(training_inputs, training_targets)

            train=int(math.ceil(len(training_inputs_array)*0.9))
            val=len(training_inputs_array)-train

            train_ds, val_ds = random_split(dataset, [train, val])

            self.train_loader = DataLoader(train_ds, self.ec_batch_size, shuffle=True)
            self.val_loader = DataLoader(val_ds, val) #will execute in 1 batch

            #Training the model
            end_epoch = self.fit()

            # #Saving the model
            # self.checkpoint = {
            #             'epoch': end_epoch + 1,
            #             'state_dict': self.classifier.state_dict(),
            #             'optimizer': self.ec_optimizer.state_dict()
            #         }

            checkpoint_dir = "entityClassifier/model_checkpoints"
            self.save_ckp(self.checkpoint, True, checkpoint_dir)

        else:
            
            # define checkpoint saved path
            ckp_path = "entityClassifier/model_checkpoints/classifier_checkpoint_model100.pt" #100

            # load the saved checkpoint
            self.classifier, self.ec_optimizer, self.start_epoch = self.load_ckp(ckp_path, self.classifier, self.ec_optimizer)

    def load_ckp(self, checkpoint_fpath, model, optimizer):
        checkpoint = torch.load(checkpoint_fpath)
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        return model, optimizer, checkpoint['epoch']

    def save_ckp(self,state, is_best, checkpoint_dir):
        f_path = checkpoint_dir + '/classifier_checkpoint_model100.pt' #100
        torch.save(state, f_path)

    def fit(self):
        # Train Network
        history_validation = []
        history_training= []
        no_improvement_counter=0
        best_loss = np.float('inf')
        best_f1 = np.float('-inf')
        for epoch in range(self.ec_num_epochs):
            training_batch_loss=[]
            for batch_idx, (data, targets) in enumerate(self.train_loader):
                # Get data to cuda if possible
                data = data.to(device=device)
                targets = targets.unsqueeze(1).to(device=device)

                # forwards
                out = self.classifier(data)

                # print('checking shapes:')
                # print(out.shape)
                # print(targets.shape)

                loss = self.ec_criterion(out, targets)
                training_batch_loss.append(loss.item())
                # print(loss.item())

                # backward
                self.ec_optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.classifier.parameters(), 1.0)
                # gradient descent or adam step
                self.ec_optimizer.step()
            combined_training_loss = np.mean(training_batch_loss)
            history_training.append(combined_training_loss)

            #Validation: DO NOT BACKPROPAGATE HERE
            validation_batch_loss = []
            labels = []
            prediction = []
            with torch.no_grad():
                for batch_idx, (val_data, val_targets) in enumerate(self.val_loader):
                    val_data = val_data.to(device=device)
                    val_targets = val_targets.unsqueeze(1).to(device=device)
                    out = self.classifier(val_data)

                    # print('checking shapes:')
                    # print(out.shape)
                    # print(val_targets.shape)
                    prediction+=out.reshape(-1).tolist()
                    labels+=val_targets

                    # loss = F.mse_loss(out, val_targets) round
                    loss = self.ec_criterion(out, val_targets)
                    validation_batch_loss.append(loss.item())
                    # print(validation_batch_loss)
                combined_validation_loss= np.mean(validation_batch_loss)

                class_prediction = [round(elem) for elem in prediction]
                assert len(class_prediction)==len(labels)
                tp = len([elem for idx, elem in enumerate(class_prediction) if((labels[idx]==1)&(elem==1))])
                fp = len([elem for idx, elem in enumerate(class_prediction) if((labels[idx]==0)&(elem==1))])
                fn = len([elem for idx, elem in enumerate(labels) if((elem==1)&(class_prediction[idx]==0))])

                precision = tp/(tp+fp)
                recall = tp/(tp+fn)
                f1 = 2*precision*recall/(precision + recall)

                history_validation.append(combined_validation_loss)
                
                print('Epoch',str(epoch+1),':',combined_training_loss,',',combined_validation_loss)
                print(len(class_prediction),len(labels))
                print('precision:',precision,'recall:',recall,'f1:',f1)
                if(((epoch+1)%10==0)|(epoch == (self.ec_num_epochs-1))):
                    print('=========')
                if(combined_validation_loss<best_loss):
                # if(f1>best_f1):
                    best_loss = combined_validation_loss
                    # best_f1 = f1
                    print('making this the checkpoint to save')
                    #Saving the model
                    self.checkpoint = {
                                'epoch': epoch + 1,
                                'state_dict': self.classifier.state_dict(),
                                'optimizer': self.ec_optimizer.state_dict()
                            }
                    no_improvement_counter=0
                else:
                    no_improvement_counter+=1
                    if(no_improvement_counter>self.patience):
                        break

        return epoch

    def run(self,candidateBase):

        candidateBase['probability']=-1
        max_length=candidateBase['length'].max()
        candidateBase['normalized_length']= candidateBase['length']/max_length
        for column in self.combined_feature_list[1:]:
            candidateBase['normalized_'+column]=candidateBase[column]/candidateBase['cumulative']

        test_inputs_array = candidateBase[self.relevant_columns].to_numpy()
        test_targets_array = candidateBase['probability'].to_numpy()

        test_inputs_array_standardized = self.scaler.fit_transform(test_inputs_array)

        test_inputs = torch.from_numpy(test_inputs_array_standardized).type(torch.float)
        # test_inputs = torch.from_numpy(test_inputs_array).type(torch.float)
        test_targets = torch.from_numpy(test_targets_array).type(torch.float)

        test_dataset = TensorDataset(test_inputs, test_targets)
        test_loader = DataLoader(test_dataset, len(test_dataset)) #will execute in 1 batch

        #Testing
        prediction=[]
        with torch.no_grad():
            for batch_idx, (data, targets) in enumerate(test_loader):
                data = data.to(device=device)
                # targets = targets.to(device=device)
                out = self.classifier(data)
                print(out.shape)
                prediction=out.reshape(-1)
                print(prediction.shape)

        candidateBase['probability'] = prediction.tolist()
        print(candidateBase['probability'].min(), candidateBase['probability'].max())
        return candidateBase

## **PHASE I: Local NER Engine (Results already returned from Gaguilar, mainly dataframe preparation for Phase II)**

In [ ]:
import re
import emoji
from emoji import demojize
import string

# import numpy as np
import pandas  as pd
from itertools import groupby
from operator import itemgetter
from collections import Iterable, OrderedDict
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import gutenberg
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer
from nltk.tokenize import TweetTokenizer

import time
import datetime
import copy
import trie as trie
import ast

import math
import shutil

cachedStopWords = stopwords.words("english")
tempList=["i","and","or","other","since","hence","onto","another","across","unlike","anytime","were","you","then","still","till","nor","perhaps","probably","otherwise","until","sometimes","sometime","seem","cannot","seems","because","can","like","into","able","unable","either","neither","if","we","it","else","elsewhere","how","not","what","who","when","where","who's","who’s","let","today","tomorrow","tonight","let's","let’s","lets","know","make","oh","via","i","yet","must","mustnt","mustn't","mustn’t","i'll","i’ll","you'll","you’ll","we'll","we’ll","done","doesnt","doesn't","doesn’t","dont","don't","don’t","did","didnt","didn't","didn’t","much","without","could","couldn't","couldn’t","would","wouldn't","wouldn’t","should","shouldn't","souldn’t","shall","isn't","isn’t","hasn't","hasn’t","wasn't","wasn’t","also","let's","let’s","let","well","just","everyone","anyone","noone","none","someone","theres","there's","there’s","everybody","nobody","somebody","anything","else","elsewhere","something","nothing","everything","i'd","i’d","i’m","won't","won’t","i’ve","i've","they're","they’re","we’re","we're","we'll","we’ll","we’ve","we've","they’ve","they've","they’d","they'd","they’ll","they'll","again","you're","you’re","you've","you’ve","thats","that's",'that’s','here’s',"here's","what's","what’s","i’m","i'm","a","so","except","arn't","aren't","arent","this","when","it","it’s","it's","he's","she's","she'd","he'd","he'll","she'll","she’ll","many","can't","cant","can’t","even","yes","no","these","here","there","to","maybe","<hashtag>","<hashtag>.","ever","every","never","there's","there’s","whenever","wherever","however","whatever","always","although"]
for item in tempList:
    if item not in cachedStopWords:
        cachedStopWords.append(item)
cachedStopWords.remove("don")
# cachedStopWords.remove("your")
# cachedStopWords.remove("up")

cachedTitles = ["mr.","mr","mrs.","mrs","miss","ms","sen.","dr","dr.","prof.","president","congressman"]
prep_list=["of","v."]  #includes common conjunction as well
# prep_list=[]
# article_list=[]
article_list=["a","an","the"]
conjoiner=["de"]
day_list=["sunday","monday","tuesday","wednesday","thursday","friday","saturday","mon","tues","wed","thurs","fri","sat","sun"]
month_list=["january","february","march","april","may","june","july","august","september","october","november","december","jan","feb","mar","apr","may","jun","jul","aug","sep","oct","nov","dec"]
chat_word_list=["nope","gee","hmm","httpurl","n't","pls","bye","€","blah","vs","ouch","am","pm","omw","http","https","tv","tmw","og","psst","b.s","thanku","em","ip","qft","ima","icymi","bdsm","ah","ive","qt","dj","dm","pts","pt","yrs","congrat","haueheuaeh","ahushaush","jr","please","retweet","2mrw","2moro","4get","ooh","reppin","idk","oops","yup","stfu","uhh","2b","dear","yay","btw","ahhh","b4","ugh","ty","cuz","coz","sorry","yea","asap","ur","bs","rt","lmfao","lfmao","slfmao","u","r","nah","umm","ummm","thank","thanks","congrats","whoa","rofl","ha","ok","okay","hey","hi","huh","ya","yep","yeah","fyi","duh","damn","lol","omg","congratulations","fucking","fuck","f*ck","wtf","wth","aka","wtaf","xoxo","rofl","imo","wow","fck","haha","hehe","hoho"]
string.punctuation=string.punctuation+'…‘’'



class LocalNERModule():
    def __init__(self, sentenceTokenizer, sentence_df_dict_gaguilar, tweet_to_sentences_w_annotation, device):
        self.counter=0

        if(sentenceTokenizer):
            self.my_sentence_tokenizer = sentenceTokenizer
        else:
            nltk.download('gutenberg')
            gutenberg_text = ""
            for file_id in gutenberg.fileids():
                gutenberg_text += gutenberg.raw(file_id)
            tokenizer_trainer = PunktTrainer()
            tokenizer_trainer.INCLUDE_ALL_COLLOCS = True
            tokenizer_trainer.train(gutenberg_text)

            self.my_sentence_tokenizer = PunktSentenceTokenizer(tokenizer_trainer.get_params())
            self.my_sentence_tokenizer._params.abbrev_types.add('dr')
            self.my_sentence_tokenizer._params.abbrev_types.add('c.j')
            self.my_sentence_tokenizer._params.abbrev_types.add('u.s')
            self.my_sentence_tokenizer._params.abbrev_types.add('u.s.a')

        self.quickRegex=re.compile("[a-z]+")

        self.tweet_to_sentences_w_annotation = tweet_to_sentences_w_annotation
        self.device = device
        self.apostrophe_list =["'s",'’s','s']
        self.label_list = ['O','B','I']

        self.tweetTokenizer = TweetTokenizer()

        self.contextual_embeddings = {}

        self.gaguilar_sentence_outputs = sentence_df_dict_gaguilar

        print('Started Local NER Engine!')

    def is_float(self,string):
        try:
            f=float(string)
            if(f==0.0):
              return True
            else:
              return ((f) and (string.count(".")==1))
      #return True# True if string is a number with a dot
        except ValueError:  # if string is not a number
          return False

    def normalize_to_sentences(self, text):
        tweetSentences=list(filter (lambda sentence: len(sentence)>1, text.split('\n')))
        tweetSentenceList_inter=self.custom_flatten(list(map(lambda sentText: self.my_sentence_tokenizer.tokenize(sentText.lstrip().rstrip()),tweetSentences)),[])
        tweetSentenceList=list(filter (lambda sentence: len(sentence)>1, tweetSentenceList_inter))
        return tweetSentenceList

    def custom_flatten(self, mylist, outlist, ignore_types=(str, bytes, int)):
        
        if (mylist !=[]):
            for item in mylist:
                #print not isinstance(item, ne.NE_candidate)
                if isinstance(item, list) and not isinstance(item, ignore_types):
                    self.custom_flatten(item, outlist)
                else:
                    item=item.strip(' \t\n\r')
                    outlist.append(item)
        return outlist

    def getWords(self, sentence):
        tempList=[]
        tempWordList=sentence.split()
        p_dots= re.compile(r'[.]{2,}')
        #print(tempWordList)
        for word in tempWordList:
            temp=[]

            if "(" in word:
                temp=list(filter(lambda elem: elem!='',word.split("(")))
                if(temp):
                    temp=list(map(lambda elem: '('+elem, temp))
            elif ")" in word:
                temp=list(filter(lambda elem: elem!='',word.split(")")))
                if(temp):
                    temp=list(map(lambda elem: elem+')', temp))
                # temp.append(temp1[-1])
    #         elif (("-" in word)&(not word.endswith("-"))):
    #             temp1=list(filter(lambda elem: elem!='',word.split("-")))
    #             if(temp1):
    #                 temp=list(map(lambda elem: elem+'-', temp1[:-1]))
    #             temp.append(temp1[-1])
            elif (("?" in word)&(not word.endswith("?"))):
                temp1=list(filter(lambda elem: elem!='',word.split("?")))
                if(temp1):
                    temp=list(map(lambda elem: elem+'?', temp1[:-1]))
                temp.append(temp1[-1])
            elif ((":" in word)&(not word.endswith(":"))):
                temp1=list(filter(lambda elem: elem!='',word.split(":")))
                if(temp1):
                    temp=list(map(lambda elem: elem+':', temp1[:-1]))
                temp.append(temp1[-1])
            elif (("," in word)&(not word.endswith(","))):
                #temp=list(filter(lambda elem: elem!='',word.split(",")))
                temp1=list(filter(lambda elem: elem!='',word.split(",")))
                if(temp1):
                    temp=list(map(lambda elem: elem+',', temp1[:-1]))
                temp.append(temp1[-1])
            elif (("/" in word)&(not word.endswith("/"))):
                temp1=list(filter(lambda elem: elem!='',word.split("/")))
                if(temp1):
                    temp=list(map(lambda elem: elem+'/', temp1[:-1]))
                temp.append(temp1[-1])
            elif (list(p_dots.finditer(word))):
                matched_spans= list(p_dots.finditer(word)) 
                temp=[]
                next_string_start=0
                for matched_span in matched_spans:
                    matched_start=matched_span.span()[0]
                    this_excerpt=word[next_string_start:matched_start]
                    if(this_excerpt):
                        temp.append(this_excerpt)
                    next_string_start=matched_span.span()[1]
                if(next_string_start<len(word)):
                    last_excerpt=word[next_string_start:]
                    if(last_excerpt):
                        temp.append(last_excerpt)
            elif "…" in word:
                temp=list(filter(lambda elem: elem!='',word.split("…")))
                if(temp):
                    if(word.endswith("…")):
                        temp=list(map(lambda elem: elem+'…', temp))
                    else:
                        temp=list(map(lambda elem: elem+'…', temp[:-1]))+[temp[-1]]
            else:
                #if word not in string.punctuation:
                temp=[word]
            if(temp):
                tempList.append(temp)
        tweetWordList=self.custom_flatten(tempList,[])
        return tweetWordList

    def rreplace(self,s, old, new, occurrence):
        if s.endswith(old):
            li = s.rsplit(old, occurrence)
            return new.join(li)
        else:
            return s

    def remAmpersand(self,candidateStr):
        candidateStr=candidateStr.replace('&amp;','')
        return candidateStr

    def normalizeToken(self,token):
        lowercased_token = token.lower()
        if token.startswith("@"):
            return "@USER"
        elif lowercased_token.startswith("http") or lowercased_token.startswith("www"):
            return "HTTPURL"
        elif len(token) == 1:
            return demojize(token)
        else:
            if token == "’":
                return "'"
            elif token == "…":
                return "..."
            else:
                return token

    def normalizeTweet(self, tweet):
        tokens = self.tweetTokenizer.tokenize(tweet.replace("’", "'").replace("…", "..."))
        normTweet = " ".join([self.normalizeToken(token) for token in tokens])

        normTweet = normTweet.replace("cannot ", "can not ").replace("n't ", " n't ").replace("n 't ", " n't ").replace("ca n't", "can't").replace("ai n't", "ain't")
        normTweet = normTweet.replace("'m ", " 'm ").replace("'re ", " 're ").replace("'s ", " 's ").replace("'ll ", " 'll ").replace("'d ", " 'd ").replace("'ve ", " 've ")
        normTweet = normTweet.replace(" p . m .", "  p.m.") .replace(" p . m ", " p.m ").replace(" a . m .", " a.m.").replace(" a . m ", " a.m ")

        normTweet = re.sub(r",([0-9]{2,4}) , ([0-9]{2,4})", r",\1,\2", normTweet)
        normTweet = re.sub(r"([0-9]{1,3}) / ([0-9]{2,4})", r"\1/\2", normTweet)
        normTweet = re.sub(r"([0-9]{1,3})- ([0-9]{2,4})", r"\1-\2", normTweet)
        
        return normTweet

    #removing commonly used expletives, enunciated chat words and other common words (like days of the week, common expressions)
    def slang_remove(self,ne_phrase):
        phrase=ne_phrase.strip().strip(string.punctuation).lower()
        p1= re.compile(r'([A-Za-z]+)\1\1{1,}')
        match_lst = p1.findall(phrase)
        if phrase in article_list:
            return True
        elif phrase in day_list:
            return True
        elif phrase in month_list:
            return True
        elif match_lst:
            return True
        else:
            return False

    def all_slang(self, ne_phrase):
        ne_words=ne_phrase.split()
        combined=[]+cachedStopWords+cachedTitles+chat_word_list+day_list
        is_invalid=0

        for word in ne_words:
            if(word.strip().strip(string.punctuation).lower() in combined):
                is_invalid+=1
        if(is_invalid==len(ne_words)):
            return True
        else:
            return False

    def start_end_token_check(self, ne_candidate):
        positions = ne_candidate[1]
        ne_words=ne_candidate[0].split()
        combined=[]+cachedStopWords+cachedTitles+chat_word_list+day_list+prep_list+article_list
        # print(combined)
        # print(ne_words,positions)
        if(ne_words):
            start_word = ne_words[0].strip().strip(string.punctuation).lower()
            end_word = ne_words[-1].strip().strip(string.punctuation).lower()
            while((len(ne_words)>1)&((start_word in combined)|(end_word in combined))):
                if(start_word in combined):
                    ne_words.pop(0)
                    positions.pop(0)
                if(end_word in combined):
                    ne_words.pop()
                    positions.pop()
                if(len(ne_words)>1):
                    start_word = ne_words[0].strip().strip(string.punctuation).lower()
                    end_word = ne_words[-1].strip().strip(string.punctuation).lower()
        ne_phrase = (' '.join(ne_words)).strip()
        # print(ne_phrase)
        return (ne_phrase,positions)

    def set_stopword_exceptions(self,words):
        combined=cachedStopWords+prep_list+article_list+day_list
        for word in words:
            if word in combined:
                self.swSet.add(word)
    
    def compile_token_embeddings(self, lst):
        token_embeddings=[]
        for tup in lst:
            token = tup[0]
            # tup[1] is feature dict, keys 0 to 99
            embedding_dict = tup[1]
            embedding_arr = np.array([embedding_dict['feat'+str(i)] for i in range(0,100)])
            token_embeddings.append(torch.from_numpy(embedding_arr).float())
        return token_embeddings

    def extract(self, batch_number):
        print("Running Local NER now")
        time_in=time.time()
        self.df_out= pd.DataFrame(columns=('tweetID', 'sentID', 'TweetSentence','tweetwordList', 'phase1Candidates','start_time','entry_batch'))
        df_holder=[]
        self.swSet= set()

        if(self.counter==0):
            #self.df_out= pd.DataFrame(columns=('tweetID', 'sentID', 'hashtags', 'user', 'TweetSentence', 'phase1Candidates','correct_candidates_tweet'))
            #dict1 = {'tweetID':0, 'sentID':0, 'hashtags':'first', 'user':'user', 'TweetSentence':'sentence', 'phase1Candidates':'phase1Out','start_time':'now','entry_batch':'batch_number'}
            self.CTrie=trie.Trie("ROOT")
            self.phase2stopWordList=[]
            self.sentenceID = 0
            self.f=0

        for key, value in self.gaguilar_sentence_outputs.items():

            now = datetime.datetime.now()
            tweetID=str(value[0])
            sentID=str(value[1])
            sentence=str(value[2])

            tweetWordList = sentence.split()
            enumerated_tweetWordList=[(token,idx) for idx,token in enumerate(tweetWordList)]

            annnotated_mentions=[]

            self.sentenceID=key

            #need both entity phrases and phrase token positions
            entities_from_sentence = value[3]
            # print(entities_from_sentence)
            phase1Out=""

            #extract sentence token level 100d embeddings
            token_embedding_list= list(value[4])
            entity_aware_embeddings = self.compile_token_embeddings(token_embedding_list)
            # for tup in lst:
            #     print(tup[0],'=>',tup[1])

            just_candidates=[]

            # place some necessary filters
            entities_from_sentence= list(filter(lambda element: not self.slang_remove(element[0]), entities_from_sentence))
            entities_from_sentence= list(map(lambda element: self.start_end_token_check(element), entities_from_sentence))
            entities_from_sentence= list(filter(lambda element: not self.all_slang(element[0]), entities_from_sentence))
            entities_from_sentence= list(filter(lambda element: len(element[0])>1, entities_from_sentence))
            entities_from_sentence= list(filter(lambda element: element[0]!='', entities_from_sentence))

            

            for candidateTuple in entities_from_sentence:
                candidateText, positions = candidateTuple
                candidateText=(((candidateText.lstrip(string.punctuation)).rstrip(string.punctuation)).strip(' \t\n\r')).lower()
                candidateText=(self.remAmpersand(candidateText).lstrip('“‘’”')).rstrip('“‘’”')
                candidateText= self.rreplace(self.rreplace(self.rreplace(candidateText,"'s","",1),"’s","",1),"’s","",1)
                candidateText= candidateText.strip()
                self.set_stopword_exceptions(candidateText.split())
                just_candidates.append(candidateText)
                # if(index==9423):
                #     print(candidateText)
                position = '*'+'*'.join(str(v) for v in positions)
                position=position+'*'

                phase1Out+=(((candidateText).lstrip(string.punctuation)).strip())+ '::'+str(position)+"||" 

                combined=[]+cachedStopWords+cachedTitles+prep_list+chat_word_list+article_list+day_list
                if not ((candidateText in combined)|(len(candidateText)<=1)|(candidateText.isdigit())|(self.is_float(candidateText))):
                    if(self.quickRegex.match(candidateText)):
                        self.CTrie.__setitem__(candidateText.split(),len(candidateText.split()),[],batch_number)
            
            # print('entities_from_sentence:',just_candidates)
            #storing the outputs and token embeddings in dataframe tweetID,sen_index
            dict1 = {'tweetID':str(tweetID), 'sentID':str(sentID), 'TweetSentence':sentence, 'tweetwordList': enumerated_tweetWordList,'phase1Candidates': just_candidates ,'phase1CandidatesWPositions':phase1Out,
                        'contextual_embeddings':entity_aware_embeddings,
                        'start_time':now,'entry_batch':batch_number}
            df_holder.append(dict1)
                    # self.contextual_embeddings[(tweetID,sen_index)] = entity_aware_embeddings

        time_out=time.time()
        self.append_rows(df_holder)
        self.phase2stopWordList=list(set(self.phase2stopWordList)|self.swSet)
        self.counter=self.counter+1
        
        #return (copy.deepcopy(self.df_out),copy.deepcopy(freqs),time_in,time_out)
        return (self.df_out,self.contextual_embeddings,self.CTrie,time_in,time_out,self.phase2stopWordList,self.tweet_to_sentences_w_annotation)

    def append_rows(self,df_holder):
    
        df = pd.DataFrame(df_holder)
        self.df_out=self.df_out.append(df)
        return

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  # Remove the CWD from sys.path while we load stuff.


## **Phase II: Global NER with Phrase Embedder to collect the Entity Embeddings**

In [ ]:
# coding: utf-8
from nltk.corpus import stopwords
import pandas  as pd
# import NE_candidate_module as ne
from nltk.tokenize import sent_tokenize, word_tokenize
import string
import copy
# import numpy
import math
from itertools import groupby
from operator import itemgetter
from collections import Iterable, OrderedDict
from scipy import stats
import emoji
import statistics
import pandas as pd
import time
import datetime
import trie as trie
import re
import ast
import pickle
import itertools
from scipy import spatial

# from sklearn.preprocessing import PolynomialFeatures
# from sklearn import linear_model
# from sklearn.cluster import KMeans, MeanShift
# from sklearn.metrics import silhouette_samples, silhouette_score
import torch

#CLASSIFIER CLASS
# import SVM as svm
# import entityClassifier as entityClassifier

cachedStopWords = stopwords.words("english")
tempList=["i","and","or","since","hence","onto","other","another","across","unlike","anytime","were","you","then","still","till","nor","perhaps","probably","otherwise","until","sometimes","sometime","seem","cannot","seems","because","can","like","into","able","unable","either","neither","if","we","it","else","elsewhere","how","not","what","who","when","where","who's","who’s","let","today","tomorrow","tonight","let's","let’s","lets","know","make","oh","via","i","yet","must","mustnt","mustn't","mustn’t","i'll","i’ll","you'll","you’ll","we'll","we’ll","done","doesnt","doesn't","doesn’t","dont","don't","don’t","did","didnt","didn't","didn’t","much","without","could","couldn't","couldn’t","would","wouldn't","wouldn’t","should","shouldn't","souldn’t","shall","isn't","isn’t","hasn't","hasn’t","wasn't","wasn’t","also","let's","let’s","let","well","just","everyone","anyone","noone","none","someone","theres","there's","there’s","everybody","nobody","somebody","anything","else","elsewhere","something","nothing","everything","i'd","i’d","i’m","won't","won’t","i’ve","i've","they're","they’re","we’re","we're","we'll","we’ll","we’ve","we've","they’ve","they've","they’d","they'd","they’ll","they'll","again","you're","you’re","you've","you’ve","thats","that's",'that’s','here’s',"here's","what's","what’s","i’m","i'm","a","so","except","arn't","aren't","arent","this","when","it","it’s","it's","he's","she's","she'd","he'd","he'll","she'll","she’ll","many","can't","cant","can’t","even","yes","no","these","here","there","to","maybe","<hashtag>","<hashtag>.","ever","every","never","there's","there’s","whenever","wherever","however","whatever","always","although"]
for item in tempList:
    if item not in cachedStopWords:
        cachedStopWords.append(item)
cachedStopWords.remove("don")
# cachedStopWords.remove("your")
# cachedStopWords.remove("us")
cachedTitles = ["mr.","mr","mrs.","mrs","miss","ms","sen.","dr","dr.","prof.","president","congressman"]
prep_list=["of","&;","v."] #includes common conjunction as well
# prep_list=[]
# article_list=[]
article_list=["a","an","the"]
conjoiner=["de"]
day_list=["sunday","monday","tuesday","wednesday","thursday","friday","saturday","mon","tues","wed","thurs","fri","sat","sun"]
month_list=["january","february","march","april","may","june","july","august","september","october","november","december","jan","feb","mar","apr","may","jun","jul","aug","sep","oct","nov","dec"]
chat_word_list=["nope","httpurl","n't","&amp;","gee","€","hmm","bye","pls","please","yrs","4get","blah","ooh","ouch","am","tv","ima","tmw","og","psst","b.s","thanku","em","qft","ip","icymi","bdsm","ah","http","https","pm","omw","pts","pt","ive","reppin","idk","oops","yup","stfu","uhh","2b","dear","yay","btw","ahhh","b4","ugh","ty","cuz","coz","sorry","yea","asap","ur","bs","rt","lmfao","lfmao","slfmao","u","r","nah","umm","ummm","thank","thanks","congrats","whoa","rofl","ha","ok","okay","hey","hi","huh","ya","yep","yeah","fyi","duh","damn","lol","omg","congratulations","fucking","fuck","f*ck","wtf","wth","aka","wtaf","xoxo","rofl","imo","wow","fck","haha","hehe","hoho"]
string.punctuation=string.punctuation+'…‘’'



class GlobalNERModule():


    def executor(self,max_batch_value,TweetBase,CTrie,phase2stopwordList,z_score_threshold,reintroduction_threshold,raw_tweets_for_others):
    # def executor(self,TweetBase,CTrie,phase2stopwordList,z_score_threshold,reintroduction_threshold,raw_tweets_for_others)


        # SET CB
        # print(phase2stopwordList)
        candidate_featureBase_DF,data_frame_holder,phase2_candidates_holder,phase2_unnormalized_candidates_holder,correction_flag = self.set_cb(TweetBase,CTrie,phase2stopwordList,z_score_threshold,reintroduction_threshold)
        candidate_featureBase_DF.to_csv("data/candidate_base_new.csv", sep=',', encoding='utf-8')
        # return #comment out if not collecting records for classifier training

        # print(self.good_candidates)

        # SET TF 
        untrashed_tweets=self.set_tf(data_frame_holder, candidate_featureBase_DF,phase2_candidates_holder,phase2_unnormalized_candidates_holder,correction_flag)

        #mark incomplete tweets
        self.set_column_for_candidates_in_incomplete_tweets(candidate_featureBase_DF,untrashed_tweets)

        # SAVE INCOMING TWEETS FOR ANNOTATION FOR OTHERS
        # self.raw_tweets_for_others=pd.concat([self.raw_tweets_for_others,raw_tweets_for_others ])

        # DROP TF
        just_converted_tweets=self.get_complete_tf(untrashed_tweets)

        #incomplete tweets at the end of current batch
        incomplete_tweets=self.get_incomplete_tf(untrashed_tweets)

        #all incomplete_tweets---> incomplete_tweets at the end of current batch + incomplete_tweets not reintroduced
        # self.incomplete_tweets=incomplete_tweets #without reintroduction--- when everything is reintroduced, just incomplete_tweets
        # self.incomplete_tweets=pd.DataFrame([], columns=['index','entry_batch', 'tweetID', 'sentID', 'hashtags', 'user', 'TweetSentence','phase1Candidates', '2nd Iteration Candidates', '2nd Iteration Candidates Unnormalized','annotation','stanford_candidates'])
        # self.incomplete_tweets=pd.concat([incomplete_tweets,self.not_reintroduced],ignore_index=True)
        self.incomplete_tweets=pd.concat([incomplete_tweets],ignore_index=True)

        print('completed tweets:',len(just_converted_tweets))
        print('incomplete tweets:',len(incomplete_tweets))


        # #recording tp, fp , f1
        # #self.accuracy_tuples_prev_batch.append((just_converted_tweets.tp.sum(), just_converted_tweets.total_mention.sum(),just_converted_tweets.fp.sum(),just_converted_tweets.fn.sum()))


        # #operations for getting ready for next batch.
        # # self.incomplete_tweets.drop('2nd Iteration Candidates', axis=1, inplace=True)
        # self.incomplete_tweets.drop(['2nd Iteration Candidates','2nd Iteration Candidates Unnormalized'], axis=1, inplace=True)
        # self.counter=self.counter+1

        self.aggregator_incomplete_tweets= self.aggregator_incomplete_tweets.append(self.incomplete_tweets)
        self.just_converted_tweets=self.just_converted_tweets.append(just_converted_tweets)

        time_out=time.time()

        self.calculate_tp_fp_f1(z_score_threshold,untrashed_tweets,raw_tweets_for_others)

        if(self.counter==(max_batch_value+1)):
            # self.just_converted_tweets.drop('2nd Iteration Candidates', axis=1, inplace=True)
            self.just_converted_tweets.drop(['2nd Iteration Candidates','2nd Iteration Candidates Unnormalized'], axis=1, inplace=True)

            print('completed tweets: ', len(self.just_converted_tweets),'incomplete tweets: ', len(self.incomplete_tweets))
            
            print(len(list(self.just_converted_tweets.columns.values)))
            print(len(list(self.incomplete_tweets.columns.values)))

            combined_frame_list=[self.just_converted_tweets, self.incomplete_tweets]
            complete_tweet_dataframe = pd.concat(combined_frame_list)

            print('final tally: ', (len(self.just_converted_tweets)+len(self.incomplete_tweets)), len(complete_tweet_dataframe))

            #to groupby tweetID and get one tuple per tweetID
            complete_tweet_dataframe_grouped_df= (complete_tweet_dataframe.groupby('tweetID', as_index=False).aggregate(lambda x: x.tolist()))
            complete_tweet_dataframe_grouped_df['tweetID']=complete_tweet_dataframe_grouped_df['tweetID'].astype(int)
            self.complete_tweet_dataframe_grouped_df_sorted=(complete_tweet_dataframe_grouped_df.sort_values(by='tweetID', ascending=True)).reset_index(drop=True)

            print(list(self.complete_tweet_dataframe_grouped_df_sorted.columns.values))


        #self.aggregator_incomplete_tweets.to_csv("all_incompletes.csv", sep=',', encoding='utf-8')


        #self.just_converted_tweets.to_csv("all_converteds.csv", sep=',', encoding='utf-8')
        #self.incomplete_tweets.to_csv("incomplete_for_last_batch.csv", sep=',', encoding='utf-8')
        return candidate_featureBase_DF, self.complete_tweet_dataframe_grouped_df_sorted,time_out 
        # return candidate_featureBase_DF, untrashed_tweets,time_out



    def __init__(self,entity_phrase_embedder,device,filename):
        self.counter=0
        self.decay_factor=2**(-1/2)
        self.decay_base_staggering=2
        self.true_positive_count=0
        self.false_positive_count=0
        self.false_negative_count=0
        self.device=device
        self.save_file= filename

        self.entity_phrase_embedder = entity_phrase_embedder

        context_feature_list=['cf_'+str(i) for i in range(100)]
        self.candidateBaseHeaders=['candidate', 'batch', 'length','cap','substring-cap','s-o-sCap','all-cap','non-cap','non-discriminative']+context_feature_list+['cumulative']

        ## When not running on a notebook
        # self.entity_classifier = entityClassifier.EntityClassifier('data/candidate_train_records_100d.csv',True,self.device)

        ################################## To do a fresh training
        self.entity_classifier = EntityClassifier('data/candidate_train_records_large_100d.csv',True,self.device)

        ################################### To Load a pre-trained model
        # self.entity_classifier = EntityClassifier('data/candidate_train_records_large_100d.csv',False,self.device)

        self.complete_tweet_dataframe_grouped_df_sorted=pd.DataFrame([], columns=['tweetID', 'TweetSentence', 'ambiguous_candidates', 'annotation', 'candidates_with_label', 'completeness', 'current_minus_entry', 'entry_batch', 'hashtags', 'index', 'only_good_candidates', 'output_mentions', 'phase1Candidates', 'sentID', 'stanford_candidates', 'user'])


    def calculate_tp_fp_f1_generic(self,raw_tweets_for_others,state_of_art):

        column_candidates_holder = raw_tweets_for_others[state_of_art].tolist()
        

        column_annot_holder= raw_tweets_for_others['mentions_other'].tolist()
        # column_annot_holder= raw_tweets_for_others['annotation_limited types'].tolist()

        all_annotations=[]
        all_mentions=[]

        true_positive_count=0
        false_positive_count=0
        false_negative_count=0
        ambigious_not_in_annotation=0
        total_mentions=0

        true_positive_holder = []
        false_negative_holder=[]
        false_positive_holder=[]
        total_mention_holder=[]
        ambigious_not_in_annotation_holder=[]
        f_measure_holder=[]

        print('========================='+state_of_art)

        total_annotation=0

        for idx in range(len(column_annot_holder)):
            unrecovered_annotated_mention_list=[]
            tp_counter_inner=0
            fp_counter_inner=0
            fn_counter_inner=0

            annotated_mention_list=[]
            tweet_level_candidate_list=column_annot_holder[idx].split(';')
            for tweet_level_candidates in tweet_level_candidate_list:
                sentence_level_cand_list= tweet_level_candidates.split(',')
                annotated_mention_list.extend(sentence_level_cand_list)
            annotated_mention_list=list(map(lambda element: element.lower().strip(),annotated_mention_list))
            annotated_mention_list=list(filter(lambda element: (element !=''), annotated_mention_list))

            if(state_of_art=='calai_candidates'):
                output_mentions_list=ast.literal_eval(column_candidates_holder[idx])
            else:
                output_mentions_list=column_candidates_holder[idx].split(',')
            output_mentions_list=list(map(lambda element: element.lower().strip(),output_mentions_list))
            output_mentions_list=list(filter(lambda element: (element !=''), output_mentions_list))

            # print(annotated_mention_list,output_mentions_list)
            all_annotations.extend(annotated_mention_list)
            all_mentions.extend(output_mentions_list)

            total_annotation+=len(annotated_mention_list)

            total_mentions+=len(output_mentions_list)
            all_postitive_counter_inner=len(output_mentions_list)

            while(annotated_mention_list):
                if(len(output_mentions_list)):
                    annotated_candidate= annotated_mention_list.pop()
                    if(annotated_candidate in output_mentions_list):
                        output_mentions_list.pop(output_mentions_list.index(annotated_candidate))
                        tp_counter_inner+=1
                    else:
                        unrecovered_annotated_mention_list.append(annotated_candidate)
                else:
                    unrecovered_annotated_mention_list.extend(annotated_mention_list)
                    break

            # unrecovered_annotated_mention_list_outer.extend(unrecovered_annotated_mention_list)
            fn_counter_inner=len(unrecovered_annotated_mention_list)
            fp_counter_inner=all_postitive_counter_inner- tp_counter_inner

            true_positive_count+=tp_counter_inner
            false_positive_count+=fp_counter_inner
            false_negative_count+=fn_counter_inner

        print(true_positive_count,false_positive_count,false_negative_count,total_mentions,total_annotation)

        # all_annotations=set(all_annotations)
        # all_mentions=set(all_mentions)

        # true_positive_count= len(all_annotations.intersection(all_mentions))
        # false_positive_count=len(all_mentions-all_annotations)
        # false_negative_count=len(all_annotations-all_mentions)
        # total_mentions=len(all_mentions)
        # total_annotation=len(all_annotations)


        # print(true_positive_count,false_positive_count,false_negative_count,total_mentions,total_annotation)

        # print(false_positive_count)
        # print(false_negative_count)
        precision=(true_positive_count)/(true_positive_count+false_positive_count)
        recall=(true_positive_count)/(true_positive_count+false_negative_count)
        f_measure=2*(precision*recall)/(precision+recall)

        if(state_of_art=="ritter_candidates"):
            self.accuracy_vals_ritter.append((f_measure,precision,recall))    
        if(state_of_art=="stanford_candidates"):
            self.accuracy_vals_stanford.append((f_measure,precision,recall))
        if(state_of_art=="calai_candidates"):
            self.accuracy_vals_opencalai.append((f_measure,precision,recall))    
        # print('z_score:', z_score_threshold , 'precision: ',precision,'recall: ',recall,'f measure: ',f_measure)
        # print('trupe positive: ',tp_count, 'false positive: ',fp_count,'false negative: ', fn_count,'total mentions: ', tm_count)

        # tweet_ids_df["tp"+state_of_art]=true_positive_holder
        # tweet_ids_df["fn"+state_of_art]=false_negative_holder
        # tweet_ids_df['fp'+state_of_art]= false_positive_holder
        
        # if(state_of_art=="ritter_candidates"):
        #     tweet_ids_df.to_csv("ritter_results.csv", sep=',', encoding='utf-8')

        # if(state_of_art=="stanford_candidates"):
        #     tweet_ids_df.to_csv("stanford_results.csv", sep=',', encoding='utf-8')

    def calculate_tp_fp_f1_alternate(self,raw_tweets_for_others, state_of_art):
        if(state_of_art=='neuroner'):
            # fp= open("/home/satadisha/Desktop/GitProjects/NeuroNER-master/neuroner/output/venezuela_input_2019-05-10_12-33-16-15380/mentions_output.txt","r")
            fp= open("/home/satadisha/Desktop/GitProjects/NeuroNER-master/neuroner/output/tweets_3K_input_2019-04-26_16-49-32-20455/mentions_output.txt","r")
            mentions_list = fp.read().split("\n") # Create a list containing all lines
            fp.close() # Close file
        if(state_of_art=='stanford_candidates'):
            fp= open("/home/satadisha/Desktop/stanford-ner-2016-10-31/stanford_venezuela_mentions.txt","r")
            mentions_list = fp.read().split("\n") # Create a list containing all lines
            fp.close() # Close file
        if(state_of_art=='ritter_candidates'):
            tweets_ritter=pd.read_csv("/home/satadisha/Desktop/GitProjects/twitter_nlp-master/ritter-venezuela-output.csv",sep =',', keep_default_na=False)
        if(state_of_art=='calai_candidates'):
            tweets_calai=pd.read_csv("/home/satadisha/Desktop/opencalai_versions/venezuela_output.csv",sep =',', keep_default_na=False)

        true_positive_count=0
        false_positive_count=0
        false_negative_count=0
        ambigious_not_in_annotation=0
        total_mentions=0

        true_positive_holder = []
        false_negative_holder=[]
        false_positive_holder=[]
        total_mention_holder=[]
        ambigious_not_in_annotation_holder=[]
        f_measure_holder=[]

        print('========================='+state_of_art)

        total_annotation=0
        all_annotations=[]
        all_mentions=[]

        for index, row in raw_tweets_for_others.iterrows():
            

            unrecovered_annotated_mention_list=[]
            tp_counter_inner=0
            fp_counter_inner=0
            fn_counter_inner=0

            annotated_mention_list=[]
            tweet_level_candidate_list=row['mentions_other'].split(';')
            # tweet_level_candidate_list=row['annotation_limited types'].split(';')
            for tweet_level_candidates in tweet_level_candidate_list:
                sentence_level_cand_list= tweet_level_candidates.split(',')
                annotated_mention_list.extend(sentence_level_cand_list)
            annotated_mention_list=list(map(lambda element: element.lower().strip(),annotated_mention_list))
            annotated_mention_list=list(filter(lambda element: (element !=''), annotated_mention_list))

            if(state_of_art=='ritter_candidates'):
                # output_mentions_list=ast.literal_eval(mentions_list[idx])
                output_mentions_list=tweets_ritter[tweets_ritter['ID']==row['ID']]['Output'].iloc[0].split(',')
            if(state_of_art=='calai_candidates'):
                output_mentions_list=tweets_calai[tweets_calai['ID']==row['ID']]['calai_candidates'].iloc[0].split(',')
            if((state_of_art=='neuroner')|(state_of_art=='stanford_candidates')):
                #for 3k Tweets:
                idx=int(row['ID'])

                # #for others:
                # idx=int(row['ID']-1)

                output_mentions_list=mentions_list[idx].split(',')

            output_mentions_list=list(map(lambda element: element.lower().strip(),output_mentions_list))
            output_mentions_list=list(filter(lambda element: (element !=''), output_mentions_list))

            print(annotated_mention_list,output_mentions_list)

            all_annotations.extend(annotated_mention_list)
            all_mentions.extend(output_mentions_list)

            total_annotation+=len(annotated_mention_list)

            total_mentions+=len(output_mentions_list)
            all_postitive_counter_inner=len(output_mentions_list)

            while(annotated_mention_list):
                if(len(output_mentions_list)):
                    annotated_candidate= annotated_mention_list.pop()
                    if(annotated_candidate in output_mentions_list):
                        output_mentions_list.pop(output_mentions_list.index(annotated_candidate))
                        tp_counter_inner+=1
                    else:
                        unrecovered_annotated_mention_list.append(annotated_candidate)
                else:
                    unrecovered_annotated_mention_list.extend(annotated_mention_list)
                    break

            # unrecovered_annotated_mention_list_outer.extend(unrecovered_annotated_mention_list)
            fn_counter_inner=len(unrecovered_annotated_mention_list)
            fp_counter_inner=all_postitive_counter_inner- tp_counter_inner

            true_positive_count+=tp_counter_inner
            false_positive_count+=fp_counter_inner
            false_negative_count+=fn_counter_inner

        print(true_positive_count,false_positive_count,false_negative_count,total_mentions,total_annotation)

        # all_annotations=set(all_annotations)
        # all_mentions=set(all_mentions)

        # true_positive_count= len(all_annotations.intersection(all_mentions))
        # false_positive_count=len(all_mentions-all_annotations)
        # false_negative_count=len(all_annotations-all_mentions)
        # total_mentions=len(all_mentions)
        # total_annotation=len(all_annotations)


        # print(true_positive_count,false_positive_count,false_negative_count,total_mentions,total_annotation)

        # print(false_positive_count)
        # print(false_negative_count)
        precision=(true_positive_count)/(true_positive_count+false_positive_count)
        recall=(true_positive_count)/(true_positive_count+false_negative_count)
        f_measure=2*(precision*recall)/(precision+recall)

        if(state_of_art=="ritter_candidates"):
            self.accuracy_vals_ritter.append((f_measure,precision,recall))    
        if(state_of_art=="stanford_candidates"):
            self.accuracy_vals_stanford.append((f_measure,precision,recall))
        if(state_of_art=="calai_candidates"):
            self.accuracy_vals_opencalai.append((f_measure,precision,recall)) 
        if(state_of_art=="neuroner"):
            self.accuracy_vals_neuroner.append((f_measure,precision,recall))


        # output_mentions_list= mentions_list[output_index].split(',')
#     # output_mentions_list=list(map(lambda element: element.lower().strip(),output_mentions_list))
#     # output_mentions_list=list(filter(lambda element: element !='', output_mentions_list))

    def calculate_tp_fp_f1_for_others(self,raw_tweets_for_others):

        opencalai="calai_candidates"
        stanford="stanford_candidates"
        ritter="ritter_candidates"
        neuroner="neuroner"

        self.calculate_tp_fp_f1_generic(raw_tweets_for_others,opencalai)
        self.calculate_tp_fp_f1_generic(raw_tweets_for_others,stanford)
        self.calculate_tp_fp_f1_generic(raw_tweets_for_others,ritter)

        # self.calculate_tp_fp_f1_alternate(raw_tweets_for_others,opencalai)
        # self.calculate_tp_fp_f1_alternate(raw_tweets_for_others,stanford)
        # self.calculate_tp_fp_f1_alternate(raw_tweets_for_others,ritter)

        self.calculate_tp_fp_f1_alternate(raw_tweets_for_others,neuroner)

    #################################
    #input candidate_feature_Base
    #output candidate_feature_Base with ["Z_score"], ["probability"],["class"]
    #################################
    def classify_candidate_base(self,z_score_threshold,candidate_featureBase_DF):

        # #filtering test set based on z_score
        # mert1=candidate_featureBase_DF['cumulative'].as_matrix()
        #frequency_array = np.array(list(map(lambda val: val[0], sortedCandidateDB.values())))
        # zscore_array1=stats.zscore(mert1)

        zscore_array1=stats.zscore(candidate_featureBase_DF['cumulative'])

        candidate_featureBase_DF['Z_ScoreUnweighted']=zscore_array1
        cumulative_threshold=1.0 #set threshold here
        z_score_threshold=candidate_featureBase_DF[candidate_featureBase_DF['cumulative']==cumulative_threshold].Z_ScoreUnweighted.tolist()[0]
        print(cumulative_threshold,z_score_threshold)
        #candidate_featureBase_DF.to_csv("cf_new_with_z_score.csv", sep=',', encoding='utf-8')

        #multi-word infrequent candidates ---> to be used for recall correction
        infrequent_candidates=candidate_featureBase_DF[(candidate_featureBase_DF['Z_ScoreUnweighted'] < z_score_threshold) & (candidate_featureBase_DF.length>1)].candidate.tolist()
        candidate_featureBase_DF = candidate_featureBase_DF[candidate_featureBase_DF['Z_ScoreUnweighted'] >= z_score_threshold]

        # # #######################with one unified classifier--- returns updated candidate_featureBase_DF with ["Z_score"], ["probability"],["class"] attributes.
        return (self.entity_classifier.run(candidate_featureBase_DF),infrequent_candidates)

        # #######################with only semantic classifier--- returns updated candidate_featureBase_DF with ["Z_score"], ["probability"],["class"] attributes.
        # return (self.entity_classifierII.run(candidate_featureBase_DF),infrequent_candidates)

        # #######################with two separate classifiers--- requires some additional lines of code
        # candidateList = candidate_featureBase_DF.candidate.tolist()
        # # print('before columns:',candidate_featureBase_DF.columns)
        # candidate_featureBase_DF.set_index("candidate", inplace=True)
        # # print('before columns:',candidate_featureBase_DF.columns)
        # print(candidate_featureBase_DF.index.name)

        # # returns updated candidate_featureBase_DF with ["Z_score"], ["probability"],["class"] attributes.
        # candidate_featureBase_DF_classifierI = self.entity_classifierI.run(copy.deepcopy(candidate_featureBase_DF))
        # candidate_featureBase_DF_classifierII = self.entity_classifierII.run(copy.deepcopy(candidate_featureBase_DF))

        # # candidate_featureBase_DF_classifierI.to_csv('classifierI.csv', sep=',', encoding='utf-8')
        # # candidate_featureBase_DF_classifierII.to_csv('classifierII.csv', sep=',', encoding='utf-8') #candidate_featureBase_DF.to_csv("candidate_base_new.csv", sep=',', encoding='utf-8')

        # final_probability_dict = {}
        # print5=0
        # for candidate in candidateList:
        #     prob1 = candidate_featureBase_DF_classifierI.loc[candidate]['probability']
        #     prob2 = candidate_featureBase_DF_classifierII.loc[candidate]['probability']
        #     final_probability = max(prob1,prob2)
        #     if(print5<5):
        #         print(candidate,prob1,prob2,final_probability)
        #         print5+=1
        #     final_probability_dict[candidate] = final_probability

        # candidate_featureBase_DF["probability"] = pd.Series(final_probability_dict)

        # candidate_featureBase_DF.reset_index(drop=False,inplace=True)
        # print('after columns:',candidate_featureBase_DF.columns)
        # print(candidate_featureBase_DF[['candidate','probability']].head(5))

        # return (candidate_featureBase_DF,infrequent_candidates)

    # recall_correction
    def set_partition_dict(self,candidate_featureBase_DF,infrequent_candidates):

        #print(list(self.partition_dict.keys()))
        ambiguous_bad_candidates=candidate_featureBase_DF[(((candidate_featureBase_DF.status=="a")|(candidate_featureBase_DF.status=="b"))&(candidate_featureBase_DF.length.astype(int)>1))]
        good_candidates=candidate_featureBase_DF[(candidate_featureBase_DF.status=="g")].candidate.tolist()
        flag1=False
        flag2=False
        if(len(ambiguous_bad_candidates)>0):
            # ambiguous_bad_candidates['max_column'] =ambiguous_bad_candidates[['cap','substring-cap','s-o-sCap','all-cap','non-cap','non-discriminative']].idxmax(axis=1) 
            # ambiguous_bad_candidates_wFilter= ambiguous_bad_candidates[ambiguous_bad_candidates.max_column=='substring-cap']

            #good_candidates=candidate_featureBase_DF[(candidate_featureBase_DF.status=="g")].candidate.tolist()
            #print(ambiguous_bad_candidates_wFilter.candidate.tolist())

            # for candidate in ambiguous_bad_candidates_wFilter.candidate.tolist():
            for candidate in ambiguous_bad_candidates.candidate.tolist():
                
                #print(candidate)
                if candidate not in self.partition_dict.keys():

                    substring_candidates=self.get_substring_candidates(candidate.split(),good_candidates)
                    if(len(substring_candidates)>0):
                        self.partition_dict[candidate]=substring_candidates

            flag1= True
        if(len(infrequent_candidates)>0):
            #print(len(ambiguous_bad_candidates_wFilter.candidate.tolist()))

            for candidate in infrequent_candidates:
                #print(candidate)
                if candidate not in self.partition_dict.keys():
                    substring_candidates=self.get_substring_candidates(candidate.split(),good_candidates)
                    if(len(substring_candidates)>0):
                        self.partition_dict[candidate]=substring_candidates
            flag2= True
        print(list(self.partition_dict.keys()))
        return (flag1|flag2)

    def get_reintroduced_tweets(self, reintroduction_threshold):
        #no reintroduction
        #no preferential selection
        print("incomplete tweets in batch: ",len(self.incomplete_tweets))
        # print(list(self.incomplete_tweets.columns.values))

        reintroduced_tweets=self.incomplete_tweets[(self.counter-self.incomplete_tweets['entry_batch'])<=reintroduction_threshold]
        self.not_reintroduced=self.incomplete_tweets[~self.incomplete_tweets.index.isin(reintroduced_tweets.index)]

        print("reintroduced tweets: ",len(reintroduced_tweets))
        # for i in range(self.counter):
        #     print('i:',len(self.incomplete_tweets[self.incomplete_tweets['entry_batch']==i]))
        return reintroduced_tweets

    def set_cb(self,TweetBase,CTrie,phase2stopwordList,z_score_threshold,reintroduction_threshold):

        #input new_tweets, z_score, Updated candidatebase of phase1
        #output candidate_featureBase_DF, Incomplete_tweets
        data_frame_holder=pd.DataFrame([], columns=['index','entry_batch','tweetID', 'sentID', 'TweetSentence','phase1Candidates', '2nd Iteration Candidates', '2nd Iteration Candidates Unnormalized'])
        phase2_candidates_holder=[]
        phase2_unnormalized_candidates_holder=[]
        df_holder=[]

        candidate_featureBase_DF,df_holder_extracted,phase2_candidates_holder_extracted,phase2_unnormalized_candidates_holder_extracted= self.extract(TweetBase,CTrie,phase2stopwordList,0)
        phase2_candidates_holder.extend(phase2_candidates_holder_extracted)
        phase2_unnormalized_candidates_holder.extend(phase2_unnormalized_candidates_holder_extracted)
        df_holder.extend(df_holder_extracted)

        # return #comment out if not collecting records for classifier training

        if((self.counter>0)&(len(self.incomplete_tweets)>0)):

            #tweet candidates for Reintroduction
            reintroduced_tweets=self.get_reintroduced_tweets(reintroduction_threshold)
            candidate_featureBase_DF,df_holder_extracted,phase2_candidates_holder_extracted,phase2_unnormalized_candidates_holder_extracted = self.extract(reintroduced_tweets,CTrie,phase2stopwordList,1)
            phase2_candidates_holder.extend(phase2_candidates_holder_extracted)
            phase2_unnormalized_candidates_holder.extend(phase2_unnormalized_candidates_holder_extracted)
            df_holder.extend(df_holder_extracted)

        #print(len(df_holder))
        data_frame_holder = pd.DataFrame(df_holder)
        #print(len(self.incomplete_tweets),len(data_frame_holder),len(candidate_featureBase_DF))
        

        #set ['probabilities'] for candidate_featureBase_DF
        candidate_featureBase_DF,self.infrequent_candidates= self.classify_candidate_base(z_score_threshold,candidate_featureBase_DF)

        # set readable labels (a,g,b) for candidate_featureBase_DF based on ['probabilities.']
        candidate_featureBase_DF=self.set_readable_labels(candidate_featureBase_DF)

        self.good_candidates=candidate_featureBase_DF[candidate_featureBase_DF.status=="g"].candidate.tolist()
        self.ambiguous_candidates=candidate_featureBase_DF[candidate_featureBase_DF.status=="a"].candidate.tolist()
        self.bad_candidates=candidate_featureBase_DF[candidate_featureBase_DF.status=="b"].candidate.tolist()

        print('good_candidates:',self.good_candidates)
        print('bad_candidates:',self.bad_candidates)
        print('ambiguous_candidates:',self.ambiguous_candidates)

        # entity_candidate_records=candidate_featureBase_DF[candidate_featureBase_DF['candidate'].isin(self.good_candidates)]
        # non_entity_candidate_records=candidate_featureBase_DF[candidate_featureBase_DF['candidate'].isin(self.bad_candidates)]
        # ambiguous_candidate_records=candidate_featureBase_DF[candidate_featureBase_DF['candidate'].isin(self.ambiguous_candidates)]

        correction_flag=self.set_partition_dict(candidate_featureBase_DF,self.infrequent_candidates)

        ambiguous_turned_good=[]
        ambiguous_turned_bad=[]
        ambiguous_remaining_ambiguous=[]
        converted_candidates=[]

        #['probability'],['a,g,b']
        return candidate_featureBase_DF,data_frame_holder,phase2_candidates_holder,phase2_unnormalized_candidates_holder,correction_flag


        #flush out completed tweets
        # input candidate base, looped over tweets (incomplete tweets+ new tweets)
        # output: incomplete tweets (a tags in it.), incomplete_tweets["Complete"]
    def set_tf(self,data_frame_holder,
            candidate_featureBase_DF,
            phase2_candidates_holder,phase2_unnormalized_candidates_holder,correction_flag):
        return self.set_completeness_in_tweet_frame(data_frame_holder,
            candidate_featureBase_DF,
            phase2_candidates_holder,phase2_unnormalized_candidates_holder,correction_flag)

    def get_incomplete_tf(self,untrashed_tweets):
        return untrashed_tweets[untrashed_tweets.completeness==False]

    def get_complete_tf(self,untrashed_tweets):
        return untrashed_tweets[untrashed_tweets.completeness==True]

    def compute_seen_tweets_so_far(self,start_batch,end_batch):
        if(start_batch==end_batch):
            sliced_seen_tweets=self.number_of_seen_tweets_per_batch[start_batch]


        sliced_seen_tweets=self.number_of_seen_tweets_per_batch[start_batch:]


        counter=0
        for elem in sliced_seen_tweets:
            counter=counter+elem

        return counter


    def rreplace(self,s, old, new, occurrence):
        if s.endswith(old):
            li = s.rsplit(old, occurrence)
            return new.join(li)
        else:
            return s
    #ME_EXTR=Mention.Mention_Extraction()


    # experiment function
    def set_x_axis(self,just_converted_tweets_for_current_batch):

        self.incomplete_tweets.to_csv("set_x_axis_debug.csv", sep=',', encoding='utf-8')

        self.incomplete_tweets['number_of_seen_tweets'] = self.incomplete_tweets['entry_batch'].apply(lambda x: self.compute_seen_tweets_so_far(x,self.counter))


        self.incomplete_tweets["entry_vs_tweet_seen_ratio"]=self.incomplete_tweets['entry_batch']/self.incomplete_tweets['number_of_seen_tweets']


        #counter_list= 
        self.incomplete_tweets["ratio_entry_vs_current"]=self.incomplete_tweets['entry_batch']/self.counter


        self.incomplete_tweets["current_minus_entry"]=self.counter-self.incomplete_tweets['entry_batch']

        just_converted_tweets_for_current_batch["current_minus_entry"]=self.counter-just_converted_tweets_for_current_batch['entry_batch']

        return just_converted_tweets_for_current_batch



    def set_column_for_candidates_in_incomplete_tweets(self,candidate_featureBase_DF,input_to_eval):

        incomplete_candidates= input_to_eval['2nd Iteration Candidates'].tolist()

        candidate_featureBase_DF= candidate_featureBase_DF.set_index('candidate')

        candidate_with_label_holder=[]
        one_level=[]
        

        for sentence_level_candidates in incomplete_candidates:

            one_level.clear()

            for candidate in sentence_level_candidates:
                if candidate.lower() in candidate_featureBase_DF.index:
                    # label=candidate_featureBase_DF.get_value(candidate.lower(),'status')
                    label=candidate_featureBase_DF.at[candidate.lower(),'status']
                    one_level.append((candidate,label))
                else:
                    one_level.append((candidate,"na"))

            candidate_with_label_holder.append(copy.deepcopy(one_level))


        input_to_eval["candidates_with_label"]=candidate_with_label_holder
        debug_candidates_label_list= input_to_eval['candidates_with_label'].tolist()
        candidates_filtered_g_labeled=[]
        row_level_candidates=[]
        index_outer=0

        candidates_filtered_a_labeled=[]
        row_level_a_candidates=[]

        for sentence_level in debug_candidates_label_list:

            # sentence_level_candidates_unnormalized= incomplete_candidates_unnormalized[index_outer]
            row_level_candidates.clear()
            row_level_a_candidates.clear()
            for candidate in sentence_level:
                if(candidate[1]=="g"):
                    candidate_str = self.erode_article(candidate[0])
                    row_level_candidates.append(candidate_str)
                if(((candidate[1]=="b")|(candidate[1]=="a"))&(candidate[0]=="US")):
                    # print('here')
                    row_level_candidates.append(candidate[0])
                if(candidate[1]=="a"):
                    row_level_a_candidates.append(candidate[0])

            candidates_filtered_g_labeled.append(copy.deepcopy(row_level_candidates))
            candidates_filtered_a_labeled.append(copy.deepcopy(row_level_a_candidates))
            index_outer+=1


        input_to_eval["only_good_candidates"]=candidates_filtered_g_labeled
        input_to_eval["ambiguous_candidates"]=candidates_filtered_a_labeled



    def calculate_tp_fp_f1(self,z_score_threshold,input_to_eval,raw_tweets_for_others):

        input_to_eval_grouped_df= (input_to_eval.groupby('tweetID', as_index=False).aggregate(lambda x: x.tolist()))
        input_to_eval_grouped_df['tweetID']=input_to_eval_grouped_df['tweetID'].astype(int)
        input_to_eval_df_sorted=(input_to_eval_grouped_df.sort_values(by='tweetID', ascending=True)).reset_index(drop=True)
        self.true_positive_count=0
        self.false_positive_count=0
        self.false_negative_count=0

        print(len(input_to_eval_df_sorted),len(raw_tweets_for_others))
        
        print(set(input_to_eval_df_sorted['tweetID'].values.tolist())-set(raw_tweets_for_others['ID'].values.tolist()))

        input_to_eval_df_sorted['annotation']=input_to_eval_df_sorted['tweetID'].apply(lambda x: raw_tweets_for_others[raw_tweets_for_others['ID']==x]['mentions_other'].iloc[0])
        # input_to_eval_df_sorted['annotation']=input_to_eval_df_sorted['tweetID'].apply(lambda x: raw_tweets_for_others[raw_tweets_for_others['ID']==x]['mentions_limited_types'].iloc[0])
        # input_to_eval_df_sorted['annotation']=input_to_eval_df_sorted['tweetID'].apply(lambda x: raw_tweets_for_others[raw_tweets_for_others['ID']==x]['annotation_limited types'].iloc[0])

        column_candidates_holder = input_to_eval_df_sorted['only_good_candidates'].tolist()
        # column_candidates_holder = input_to_eval_df_sorted['phase1Candidates'].tolist()

        file1 = open(self.save_file+".txt", "w")
        

        column_annot_holder= input_to_eval_df_sorted['annotation'].tolist()
        
        output_str=''
        # print(column_candidates_holder)

        true_positive_count=0
        false_positive_count=0
        false_negative_count=0
        ambigious_not_in_annotation=0
        total_mentions=0
        total_annotation=0

        all_annotations=[]
        all_mentions=[]

        true_positive_holder = []
        false_negative_holder=[]
        false_positive_holder=[]
        total_mention_holder=[]
        ambigious_not_in_annotation_holder=[]
        f_measure_holder=[]

        quickRegex=re.compile("[a-z]+")

        print('=========================Gaguilar_candidates')

        for idx in range(len(column_annot_holder)):
            unrecovered_annotated_mention_list=[]
            tp_counter_inner=0
            fp_counter_inner=0
            fn_counter_inner=0

            annotated_mention_list=[]
            output_mentions_list=[]
            tweet_level_candidate_list=str(column_annot_holder[idx]).split(';')
            for tweet_level_candidates in tweet_level_candidate_list:
                sentence_level_cand_list= tweet_level_candidates.split(',')
                annotated_mention_list.extend(sentence_level_cand_list)
            annotated_mention_list=list(map(lambda element: element.lower().strip(),annotated_mention_list))
            annotated_mention_list=list(filter(lambda element: quickRegex.match(element), annotated_mention_list))
            annotated_mention_list=list(filter(lambda element: ((element !='')&(element !='nan')), annotated_mention_list))

            for lst in column_candidates_holder[idx]:
                output_mentions_list.extend(lst)

            output_mentions_list=list(filter(lambda element: (element !=''), output_mentions_list))
            output_mentions_list=list(map(lambda element: element.lower(), output_mentions_list))
            total_annotation+=len(annotated_mention_list)

            print(idx, annotated_mention_list,output_mentions_list)
            output_str+=','.join(output_mentions_list)+'\n'

            all_annotations.extend(annotated_mention_list)
            all_mentions.extend(output_mentions_list)

            total_mentions+=len(output_mentions_list)
            all_postitive_counter_inner=len(output_mentions_list)

            while(annotated_mention_list):
                if(len(output_mentions_list)):
                    annotated_candidate= self.normalize(annotated_mention_list.pop())
                    if(annotated_candidate in output_mentions_list):
                        output_mentions_list.pop(output_mentions_list.index(annotated_candidate))
                        tp_counter_inner+=1
                    else:
                        unrecovered_annotated_mention_list.append(annotated_candidate)
                else:
                    unrecovered_annotated_mention_list.extend(annotated_mention_list)
                    break

            # unrecovered_annotated_mention_list_outer.extend(unrecovered_annotated_mention_list)
            fn_counter_inner=len(unrecovered_annotated_mention_list)
            fp_counter_inner=all_postitive_counter_inner- tp_counter_inner

            print(tp_counter_inner,fp_counter_inner,fn_counter_inner)

            self.true_positive_count+=tp_counter_inner
            self.false_positive_count+=fp_counter_inner
            self.false_negative_count+=fn_counter_inner

            print(self.true_positive_count,self.false_positive_count,self.false_negative_count)

        print('TP||||FP||||FN')
        print(self.true_positive_count,self.false_positive_count,self.false_negative_count,total_mentions,total_annotation)

        precision=(self.true_positive_count)/(self.true_positive_count+self.false_positive_count)
        recall=(self.true_positive_count)/(self.true_positive_count+self.false_negative_count)
        f_measure=2*(precision*recall)/(precision+recall)

        # all_annotations=set(all_annotations)
        # all_mentions=set(all_mentions)
        
        # true_positive_count= len(all_annotations.intersection(all_mentions))
        # false_positive_count=len(all_mentions-all_annotations)
        # false_negative_count=len(all_annotations-all_mentions)
        # total_mentions=len(all_mentions)
        # total_annotation=len(all_annotations)


        # print(true_positive_count,false_positive_count,false_negative_count,total_mentions,total_annotation)


        # true_positive_count_IPQ=true_positive_count
        # false_positive_count_IPQ = false_positive_count
        # false_negative_count_IPQ= false_negative_count
        # total_mention_count_IPQ=total_mentions


        # tp_count=0
        # tm_count=0
        # fp_count=0
        # fn_count=0

        # for idx,tup in enumerate(self.accuracy_tuples_prev_batch):
        #     # print(idx,tup)
        #     tp_count+=tup[0]
        #     tm_count+=tup[1]
        #     fp_count+=tup[2]
        #     fn_count+=tup[3]



        # tp_count+=true_positive_count_IPQ
        # tm_count+=total_mention_count_IPQ
        # fp_count+=false_positive_count_IPQ
        # fn_count+=false_negative_count_IPQ

        # precision=(true_positive_count)/(true_positive_count+false_positive_count)
        # recall=(true_positive_count)/(true_positive_count+false_negative_count)
        # f_measure=2*(precision*recall)/(precision+recall)

        file1.write(output_str)
        file1.close()



        self.accuracy_vals=(z_score_threshold,f_measure,precision,recall)

        print('Precision:',precision)
        print('Recall:',recall)
        print('F1:',f_measure)

        # print('z_score:', z_score_threshold , 'precision: ',precision,'recall: ',recall,'f measure: ',f_measure)
        # print('trupe positive: ',tp_count, 'false positive: ',fp_count,'false negative: ', fn_count,'total mentions: ', tm_count)


        # input_to_eval["tp"]=true_positive_holder
        # input_to_eval["fn"]=false_negative_holder
        # input_to_eval['fp']= false_positive_holder
        # input_to_eval["total_mention"]=total_mention_holder

        # input_to_eval["ambigious_not_in_annot"]=ambigious_not_in_annotation_holder
        # input_to_eval["inverted_loss"]=input_to_eval["tp"]/( input_to_eval["fn"]+input_to_eval["ambigious_not_in_annot"])

        return input_to_eval


    def recall_correction(self,candidate_featureBase_DF,phase2_candidates_holder,phase2_unnormalized_candidates_holder,data_frame_holder):

        corrected_phase2_candidates_holder=[]
        index_outer=0
        for candidates in phase2_candidates_holder:
            unnormalized_candidates=phase2_unnormalized_candidates_holder[index_outer]
            corrected_phase2_candidates=[]
            for idx, candidate in enumerate(candidates):
                unnormalized_candidate=unnormalized_candidates[idx]
                # if((candidate in self.partition_dict.keys())&((candidate in self.infrequent_candidates)|(candidate in self.bad_candidates))):
                if((candidate in self.partition_dict.keys())&((candidate in self.infrequent_candidates)|(candidate in self.bad_candidates)|(candidate in self.ambiguous_candidates))):   #do this only for 3K tweets
                    #print(candidate, self.partition_dict[candidate])
                    corrected_phase2_candidates.extend(self.partition_dict[candidate])
                else:
                    if(((candidate in self.bad_candidates)|(candidate in self.ambiguous_candidates))&(candidate=='us')&(unnormalized_candidate=='US')):
                        # print(index_outer)
                        candidate=unnormalized_candidate
                    # if((len(candidate.strip().strip(string.punctuation).split())>1)&(candidate.strip().strip(string.punctuation).split()[0].lower() in ['a','an','the'])):
                    #     candidate = (' '.join(candidate.strip().strip(string.punctuation).split()[1:])).strip()
                    corrected_phase2_candidates.append(candidate)
            corrected_phase2_candidates_holder.append(copy.deepcopy(corrected_phase2_candidates))
            index_outer+=1

        
        #print(corrected_phase2_candidates_holder)
        data_frame_holder['2nd Iteration Candidates']=corrected_phase2_candidates_holder

        return corrected_phase2_candidates_holder,data_frame_holder                  


    def erode_article(self, entity_string):
        if((len(entity_string.strip().strip(string.punctuation).split())>1)&(entity_string.lower().strip().strip(string.punctuation).split()[0] in ['a','an','the'])):
            # print(entity_string)
            entity_string = ' '.join(entity_string.strip().strip(string.punctuation).split()[1:])
            # print(entity_string)
        return entity_string.strip()


    def set_completeness_in_tweet_frame(self,data_frame_holder,candidate_featureBase_DF,phase2_candidates_holder,phase2_unnormalized_candidates_holder,correction_flag):
        #print(candidate_featureBase_DF.head())
        good_candidates=candidate_featureBase_DF[candidate_featureBase_DF.status=="g"].candidate.tolist()
        bad_candidates=candidate_featureBase_DF[candidate_featureBase_DF.status=="b"].candidate.tolist()

        merged_g_b= bad_candidates+good_candidates

        #candidate_featureBase_DF.to_csv("cf_before_labeling_comp.csv", sep=',', encoding='utf-8')
        ambiguous_candidates=candidate_featureBase_DF[candidate_featureBase_DF.status=="a"].candidate.tolist()

        if(correction_flag):
            phase2_candidates_holder,data_frame_holder=self.recall_correction(candidate_featureBase_DF,phase2_candidates_holder,phase2_unnormalized_candidates_holder,data_frame_holder)

         

        
        truth_vals=[False if any(x not in merged_g_b for x in list1) else True for list1 in phase2_candidates_holder]

        intermediate_output_mentions=[list(filter(lambda candidate: ((candidate in good_candidates))|(candidate=='US'), list1)) for list1 in phase2_candidates_holder]

        output_mentions=[list(map(lambda candidate: self.erode_article(candidate), list1)) for list1 in intermediate_output_mentions]

        # truth_vals=[False if any(x in ambiguous_candidates for x in list1) else True for list1 in phase2_candidates_holder]

        # for list1 in phase2_candidates_holder:
        #     if any(x in ambiguous_candidates  for x in list1):
        #         truth_vals.append(False)
        #     else:
        #         truth_vals.append(True)
 


        #print(truth_vals)
        completeness_series = pd.Series( (v for v in truth_vals) )
        output_mentions_series = pd.Series( (v for v in output_mentions) )


        data_frame_holder['output_mentions']=output_mentions_series
        data_frame_holder['completeness']=completeness_series
        data_frame_holder["current_minus_entry"]=self.counter-data_frame_holder['entry_batch']

        return data_frame_holder




    def set_readable_labels(self,candidate_featureBase_DF):

        #candidate_featureBase_DF['status'] = candidate_featureBase_DF['probability'].apply(lambda x: set(x).issubset(good_candidates))
        candidate_featureBase_DF['status']='ne'
        candidate_featureBase_DF['status'][candidate_featureBase_DF['probability']>=0.50]='g'
        candidate_featureBase_DF['status'][(candidate_featureBase_DF['probability'] > 0.4) & (candidate_featureBase_DF['probability'] < 0.50)] = 'a'
        candidate_featureBase_DF['status'][candidate_featureBase_DF['probability']<=0.4]='b'

        return candidate_featureBase_DF



    def normalize(self,word):
        strip_op=word
        strip_op=(((strip_op.lstrip(string.punctuation)).rstrip(string.punctuation)).strip()).lower()
        strip_op=(strip_op.lstrip('“‘’”')).rstrip('“‘’”')
        strip_op=(((strip_op.lstrip(string.punctuation)).rstrip(string.punctuation)).strip()).lower()
        #strip_op= self.rreplace(self.rreplace(self.rreplace(strip_op,"'s","",1),"’s","",1),"’s","",1)
        if strip_op.endswith("'s"):
            li = strip_op.rsplit("'s", 1)
            return ''.join(li)
        elif strip_op.endswith("’s"):
            li = strip_op.rsplit("’s", 1)
            return ''.join(li)
        else:
            return strip_op
        #return strip_op

    
    def isSubstring(self,to_increase_element,id_to_incr,comparison_holder,phase1_holder_holder_copy):
        combined_list=comparison_holder[id_to_incr]+phase1_holder_holder_copy[id_to_incr]

        for idx,val in enumerate(comparison_holder[id_to_incr]):
            if((to_increase_element[0] in val[0]) and to_increase_element[0] != val[0]):
                if((to_increase_element[5] in val[5]) and to_increase_element[5] != val[5]):
                    return True
        for idx,val in enumerate(phase1_holder_holder_copy[id_to_incr]):
            if((to_increase_element[0] in val[0]) and to_increase_element[0] != val[0]):
                if((to_increase_element[5] in val[2]) and to_increase_element[5] != val[2]):
                    return True   
                
        return False


    def calculate_pmi(self,big,x1,x2,total):
        big__= float(big/total)
        x1__=float(x1/total)
        x2__=float(x2/total)
        pmi= math.log(big__/(x1__*x2__),2.71828182845)
        pklv=big__*pmi
        #return (1/(1+math.exp(-1*pmi)))
        npmi= pmi/(-1.0*(math.log(big__,2.71828182845)))
        return npmi,pklv
        #return pklv

    def multiSlice(self,s,cutpoints,good_candidates):
        k = len(cutpoints)
        multislices=[]
        if k == 0:
            curr_candidate=self.normalize(' '.join(s))

            if(curr_candidate in good_candidates):
                multislices = [curr_candidate]        
        else:
            
            curr_candidate=self.normalize(' '.join(s[:cutpoints[0]]))
            alt_list=[curr_candidate]
            
            if(curr_candidate in good_candidates):
                multislices = [curr_candidate]

            alt_list.extend(self.normalize(' '.join(s[cutpoints[i]:cutpoints[i+1]])) for i in range(k-1))
            multislices.extend(self.normalize(' '.join(s[cutpoints[i]:cutpoints[i+1]])) for i in range(k-1) if self.normalize(' '.join(s[cutpoints[i]:cutpoints[i+1]])) in good_candidates)

            curr_candidate=self.normalize(' '.join(s[cutpoints[k-1]:]))
            alt_list.append(curr_candidate)
            
            if(curr_candidate in good_candidates):
                multislices.append(curr_candidate)
            # print('::',alt_list)
        return multislices



    def get_substring_candidates(self,candidate_words,good_candidates):
        n = len(candidate_words)
        all_partitions=[]
        all_partitions_length=[]
        cuts = list(range(1,n))
        for k in range(n):
            # all_partitions_inner=[]
            partition_list=[]
            partition_length_list=[]
            for cutpoints in itertools.combinations(cuts,k):
                ret_list=self.multiSlice(candidate_words,cutpoints,good_candidates)
                if(ret_list):
                    partition_length=sum([len(elem.split()) for elem in ret_list])
                    # print('==',ret_list,partition_length)
                    if(partition_length==len(candidate_words)):
                        return ret_list
                    partition_list.append(ret_list)
                    partition_length_list.append(partition_length)
                    # yield ret_list
            # print('------')
            if(partition_length_list):
                max_index=partition_length_list.index(max(partition_length_list))
                all_partitions.append(partition_list[max_index])
                all_partitions_length.append(partition_length_list[max_index])
        # print(all_partitions)
        if(all_partitions_length):
            max_index=all_partitions_length.index(max(all_partitions_length))
            # print(all_partitions[max_index])
            return all_partitions[max_index]
        else:
            return []
    

    def verify(self, subsequence, CTrie):
        return CTrie.__contains__(subsequence)



    def check_sequence(self, sequence, l, CTrie):
        result=[]
        subsequence_length=l
        while(subsequence_length>0):
            shift=len(sequence)-subsequence_length
            verified_subsequence=[]
            verified=False
            for i in range(0,shift+1):
                list1=sequence[i:(i+subsequence_length)]
                text=' '.join(str(e[0]) for e in list1)
                subsequence=(self.normalize(text)).split()
                #print("search for", subsequence)
                if self.verify(subsequence, CTrie):
                    verified_subsequence.append(i)
                    verified_subsequence.append(i+subsequence_length)
                    #print(subsequence)
                    #print(subsequence,[(verified_subsequence[0]-0),(int(sequence[-1][1])-verified_subsequence[1])])
                    verified=True
                    break
            if(verified):
                result.append(sequence[verified_subsequence[0]:verified_subsequence[1]])
                if(verified_subsequence[0]-0)>0:
                    subequence_to_check=sequence[0:verified_subsequence[0]]
                    #since tokens before the starting position of the verified subsequence have already been checked for subsequences of this length
                    partition_length=min(len(subequence_to_check),(subsequence_length-1))
                    #print(subequence_to_check)
                    lst=self.check_sequence(subequence_to_check,partition_length, CTrie)
                    if(lst):
                        result.extend(lst)
                if(int(sequence[-1][1])-verified_subsequence[1])>0:
                    subequence_to_check=sequence[(verified_subsequence[1]):]
                    #since tokens following the end position of the verified subsequence have not been checked for subsequences of this length
                    partition_length=min(len(subequence_to_check),(subsequence_length))
                    #print(subequence_to_check)
                    lst=self.check_sequence(subequence_to_check,partition_length, CTrie)
                    if(lst):
                        result.extend(lst)
                return result
            else:
                subsequence_length-=1
        return result

    # def flatten(self,mylist, outlist,ignore_types=(str, bytes, int, ne.NE_candidate)):
    def flatten(self,mylist, outlist,ignore_types=(str, bytes, int)):
    
        if mylist !=[]:
            for item in mylist:
                #print not isinstance(item, ne.NE_candidate)
                if isinstance(item, list) and not isinstance(item, ignore_types):
                    self.flatten(item, outlist)
                else:
                    # if isinstance(item,ne.NE_candidate):
                    #     item.phraseText=item.phraseText.strip(' \t\n\r')
                    #     item.reset_length()
                    # else:
                    if type(item)!= int:
                        item=item.strip(' \t\n\r')
                    outlist.append(item)
        return outlist


    def getWords(self, sentence):
        tempList=[]
        tempWordList=sentence.split()
        p_dots= re.compile(r'[.]{2,}')
        #print(tempWordList)
        for word in tempWordList:
            temp=[]
            
            if "(" in word:
                temp=list(filter(lambda elem: elem!='',word.split("(")))
                if(temp):
                    temp=list(map(lambda elem: '('+elem, temp))
            elif ")" in word:
                temp=list(filter(lambda elem: elem!='',word.split(")")))
                if(temp):
                    temp=list(map(lambda elem: elem+')', temp))
                # temp.append(temp1[-1])
            # elif (("-" in word)&(not word.endswith("-"))):
            #     temp1=list(filter(lambda elem: elem!='',word.split("-")))
            #     if(temp1):
            #         temp=list(map(lambda elem: elem+'-', temp1[:-1]))
            #     temp.append(temp1[-1])
            elif (("?" in word)&(not word.endswith("?"))):
                temp1=list(filter(lambda elem: elem!='',word.split("?")))
                if(temp1):
                    temp=list(map(lambda elem: elem+'?', temp1[:-1]))
                temp.append(temp1[-1])
            elif ((":" in word)&(not word.endswith(":"))):
                temp1=list(filter(lambda elem: elem!='',word.split(":")))
                if(temp1):
                    temp=list(map(lambda elem: elem+':', temp1[:-1]))
                temp.append(temp1[-1])
            elif (("," in word)&(not word.endswith(","))):
                #temp=list(filter(lambda elem: elem!='',word.split(",")))
                temp1=list(filter(lambda elem: elem!='',word.split(",")))
                if(temp1):
                    temp=list(map(lambda elem: elem+',', temp1[:-1]))
                temp.append(temp1[-1])
            elif (("/" in word)&(not word.endswith("/"))):
                temp1=list(filter(lambda elem: elem!='',word.split("/")))
                if(temp1):
                    temp=list(map(lambda elem: elem+'/', temp1[:-1]))
                temp.append(temp1[-1])
                #print(index, temp)
            # elif "..." in word:
            #     #print("here")
            #     temp=list(filter(lambda elem: elem!='',word.split("...")))
            #     if(temp):
            #         if(word.endswith("...")):
            #             temp=list(map(lambda elem: elem+'...', temp))
            #         else:
            #            temp=list(map(lambda elem: elem+'...', temp[:-1]))+[temp[-1]]
            #     # temp.append(temp1[-1])
            # elif ".." in word:
            #     temp=list(filter(lambda elem: elem!='',word.split("..")))
            #     if(temp):
            #         if(word.endswith("..")):
            #             temp=list(map(lambda elem: elem+'..', temp))
            #         else:
            #             temp=list(map(lambda elem: elem+'..', temp[:-1]))+[temp[-1]]
            #     #temp.append(temp1[-1])
            elif (list(p_dots.finditer(word))):
                matched_spans= list(p_dots.finditer(word)) 
                temp=[]
                next_string_start=0
                for matched_span in matched_spans:
                    matched_start=matched_span.span()[0]
                    this_excerpt=word[next_string_start:matched_start]
                    if(this_excerpt):
                        temp.append(this_excerpt)
                    next_string_start=matched_span.span()[1]
                if(next_string_start<len(word)):
                    last_excerpt=word[next_string_start:]
                    if(last_excerpt):
                        temp.append(last_excerpt)
            elif "…" in word:
                temp=list(filter(lambda elem: elem!='',word.split("…")))
                if(temp):
                    if(word.endswith("…")):
                        temp=list(map(lambda elem: elem+'…', temp))
                    else:
                        temp=list(map(lambda elem: elem+'…', temp[:-1]))+[temp[-1]]
            else:
                #if word not in string.punctuation:
                temp=[word]
            if(temp):
                tempList.append(temp)
        tweetWordList=self.flatten(tempList,[])
        return tweetWordList

    def get_Candidates(self, sequence, CTrie,flag):
        #flag: debug_flag
        candidateList=[]
        left=0
        start_node=CTrie
        last_cand="NAN"
        last_cand_substr=""
        reset=False
        right=0
        while (right < len(sequence)):
            # if(flag):
            #     print(right)
            if(reset):
                start_node=CTrie
                last_cand_substr=""
                left=right
            curr_text=sequence[right][0]
            curr_pos=[sequence[right][1]]
            #normalized curr_text
            curr=self.normalize(sequence[right][0])
            cand_str=self.normalize(last_cand_substr+" "+curr)
            cand_str_wPunct=(last_cand_substr+" "+curr_text).lower()
            last_cand_sequence=sequence[left:(right+1)]
            last_cand_text=' '.join(str(e[0]) for e in last_cand_sequence)
            last_cand_text_norm=self.normalize(' '.join(str(e[0]) for e in last_cand_sequence))
            if(flag):
                print("==>",cand_str,last_cand_text_norm)
            if((cand_str==last_cand_text_norm)&((curr in start_node.path.keys())|(curr_text.lower() in start_node.path.keys()))):
            #if (((curr in start_node.path.keys())&(cand_str==last_cand_text_norm))|(curr_text.lower() in start_node.path.keys())):
                if flag:
                    print("=>",cand_str,last_cand_text)
                reset=False
                if (curr_text.lower() in start_node.path.keys()):
                    if (start_node.path[curr_text.lower()].value_valid):
                        last_cand_pos=[e[1] for e in last_cand_sequence]
                        last_cand_batch=start_node.path[curr_text.lower()].feature_list[-1]
                        last_cand=last_cand_text
                    elif(curr in start_node.path.keys()):
                        if ((start_node.path[curr].value_valid)):
                            last_cand_pos=[e[1] for e in last_cand_sequence]
                            last_cand=last_cand_text
                            last_cand_batch=start_node.path[curr].feature_list[-1]
                        else:
                            if((right==(len(sequence)-1))&(last_cand=="NAN")&(left<right)):
                                #print("hehe",cand_str)
                                right=left
                                reset=True
                    else:
                        if((right==(len(sequence)-1))&(last_cand=="NAN")&(left<right)):
                            #print("hehe",cand_str)
                            right=left
                            reset=True
                elif ((start_node.path[curr].value_valid)&(cand_str==last_cand_text_norm)):
                    # if flag:
                    #     print("==",last_cand_text)
                    last_cand_pos=[e[1] for e in last_cand_sequence]
                    last_cand=last_cand_text
                    last_cand_batch=start_node.path[curr].feature_list[-1]
                else:
                    if((right==(len(sequence)-1))&(last_cand=="NAN")&(left<right)):
                        #print("hehe",cand_str)
                        right=left
                        reset=True
                if((curr_text.lower() in start_node.path.keys())&(cand_str==last_cand_text_norm)):
                    start_node=start_node.path[curr_text.lower()]
                    last_cand_substr=cand_str_wPunct
                else:
                    start_node=start_node.path[curr]
                    last_cand_substr=cand_str
            else:
                #print("=>",cand_str,last_cand_text)
                if(last_cand!="NAN"):
                    candidateList.append((last_cand,last_cand_pos,last_cand_batch))
                    last_cand="NAN"
                    if(start_node!=CTrie):
                        start_node=CTrie
                        last_cand_substr=""
                        if curr in start_node.path.keys():
                            # if(flag):
                            #     print("here",curr)
                            reset=False
                            if start_node.path[curr].value_valid:
                                last_cand_text=curr_text
                                last_cand_pos=curr_pos
                                last_cand=last_cand_text
                                last_cand_batch=start_node.path[curr].feature_list[-1]
                            left=right
                            start_node=start_node.path[curr]
                            last_cand_substr=curr
                        else:
                            reset=True
                    else:
                        reset=True
                else:
                    if(left<right):
                        # if(flag):
                        #     print(sequence[(left+1):(right+1)])
                        #candidateList.extend(self.get_Candidates(sequence[(left+1):(right+1)], CTrie, flag))
                        right=left
                        # if(flag):
                        #     print("++",right)
                    reset=True
            right+=1
        # if(flag):
        #     print(last_cand)
        if(last_cand!="NAN"):
            candidateList.append((last_cand,last_cand_pos,last_cand_batch))
        return candidateList


    def append_rows(self,df_holder):
    
        df = pd.DataFrame(df_holder)
        #self.data_frame_holder=self.data_frame_holder.append(df,ignore_index=True)
        #self.data_frame_holder=self.data_frame_holder.reset_index(drop=True)
        return df




    def join_token_tuples(self,list_of_tuples):
        #print(string.punctuation)
        combined_str=(' '.join(tuple[0] for tuple in list_of_tuples)).lstrip(string.punctuation).rstrip(string.punctuation).strip()
        combined_pos='*'.join(str(tuple[1]) for tuple in list_of_tuples)
        combined_tuple=(combined_str,combined_pos,list_of_tuples[0][2],list_of_tuples[0][3],list_of_tuples[0][4],list_of_tuples[0][5],list_of_tuples[0][6])
        return combined_tuple




    def all_capitalized(self,candidate):
        strip_op=candidate
        strip_op=(((strip_op.lstrip(string.punctuation)).rstrip(string.punctuation)).strip())
        strip_op=(strip_op.lstrip('“‘’”')).rstrip('“‘’”')
        strip_op= self.rreplace(self.rreplace(self.rreplace(strip_op,"'s","",1),"’s","",1),"’s","",1)
        prep_article_list=prep_list+article_list+self.phase2stopwordList
        word_list=strip_op.split()
        for i in range(len(word_list)):
            word=word_list[i]
            if((word[0].isupper())|(word[0].isdigit())):
                continue
            else:
                if(word in prep_article_list):
                    if (i!=0):
                        continue
                    else:
                        return False
                elif(word in conjoiner):
                    continue
                else:
                    return False
        return True




    def check_feature_update(self, candidate_tuple,non_discriminative_flag):
        #print(candidate_tuple)
        if(non_discriminative_flag):
            return 7
        candidateText=candidate_tuple[0]
        position=candidate_tuple[1]
        word_list=candidateText.split()
        if candidateText.islower():
            return 6
        elif candidateText.isupper():
            return 5
        elif (len(word_list)==1):
            #start-of-sentence-check
            if self.all_capitalized(candidateText):
                if(int(position[0])==0):
                    return 4
                else:
                    return 2
            else:
                return 3
        else:
            if(self.all_capitalized(candidateText)):
                return 2
            else:
                return 3


    def update_Candidatedict(self,candidate_tuple,non_discriminative_flag,contextual_embedding_vector):
        candidateText=candidate_tuple[0]
        normalized_candidate=self.normalize(candidateText)
        # print('adding:',normalized_candidate)

        feature_list=[]
        if(normalized_candidate in self.CandidateBase_dict.keys()):
            feature_list=self.CandidateBase_dict[normalized_candidate]
        else:
            # feature_list=[0]*9 # only syntax
            feature_list=[0]*109 # context embedding: 100

            feature_list[0]=self.counter
            feature_list[1]=len(normalized_candidate.split())

        #syntax_feature to update
        feature_to_update=self.check_feature_update(candidate_tuple,non_discriminative_flag)
        feature_list[feature_to_update]+=1

        # add up the context embedding features
        # print(len(contextual_embedding_vector))
        feature_list[8:-1]= np.add(feature_list[8:-1],contextual_embedding_vector).tolist()

        #increment cumulative frequency
        feature_list[-1]+=1
        self.CandidateBase_dict[normalized_candidate]=feature_list





    def extract(self,tweetBaseInput,CTrie,phase2stopwordList,new_or_old):


        if(self.counter==0):
            #output_queue
            self.data_frame_holder_OQ=pd.DataFrame([], columns=['index', 'entry_batch', 'tweetID', 'sentID', 'TweetSentence','phase1Candidates', '2nd Iteration Candidates', '2nd Iteration Candidates Unnormalized'])
            self.incomplete_tweets=pd.DataFrame([], columns=['index','entry_batch', 'tweetID', 'sentID', 'TweetSentence','phase1Candidates', '2nd Iteration Candidates', '2nd Iteration Candidates Unnormalized'])
            self.not_reintroduced=pd.DataFrame([], columns=['index','entry_batch', 'tweetID', 'sentID', 'TweetSentence','phase1Candidates', '2nd Iteration Candidates', '2nd Iteration Candidates Unnormalized'])
            self.CandidateBase_dict= {}
            self.ambiguous_candidate_distanceDict_prev={}
            self.partition_dict={}
            self.good_candidates=[]
            self.bad_candidates=[]
            self.ambiguous_candidates=[]

            self.aggregator_incomplete_tweets=pd.DataFrame([], columns=['index', 'entry_batch', 'tweetID', 'sentID', 'TweetSentence','phase1Candidates', '2nd Iteration Candidates', '2nd Iteration Candidates Unnormalized'])
            self.just_converted_tweets=pd.DataFrame([], columns=['index', 'entry_batch', 'tweetID', 'sentID', 'TweetSentence','phase1Candidates', '2nd Iteration Candidates', '2nd Iteration Candidates Unnormalized'])
            #self.data_frame_holder=pd.DataFrame([], columns=['index','entry_batch','tweetID', 'sentID', 'TweetSentence','phase1Candidates', '2nd Iteration Candidates'])
            self.raw_tweets_for_others=pd.DataFrame([], columns=['index','entry_batch','tweetID', 'sentID', 'TweetSentence','phase1Candidates', '2nd Iteration Candidates', '2nd Iteration Candidates Unnormalized'])

            self.accuracy_tuples_prev_batch=[]
            self.accuracy_vals=[]
            
            #frequency_w_decay related information
            self.ambiguous_candidates_reintroduction_dict={}

            #### other systems
            self.accuracy_vals_stanford=[]
            self.accuracy_vals_opencalai=[]
            self.accuracy_vals_ritter=[]
            self.accuracy_vals_neuroner=[]

            self.number_of_seen_tweets_per_batch=[]
        self.phase2stopwordList=phase2stopwordList
        self.number_of_seen_tweets_per_batch.append(len(tweetBaseInput))


        #data_frame_holder=pd.DataFrame([], columns=['index','entry_batch','tweetID', 'sentID', 'hashtags', 'user', 'TweetSentence','phase1Candidates', '2nd Iteration Candidates', '2nd Iteration Candidates Unnormalized'])
        phase1_holder_holder=[]
        phase2_candidates_holder=[]
        phase2_unnormalized_candidates_holder=[]
        df_holder=[]
        
        #candidateBase_holder=[]

        #this has to be changed to an append function since IPQ already has incomplete tweets from prev batch  
        #print(len(tweetBaseInput))
        #immediate_processingQueue = pd.concat([self.incomplete_tweets,TweetBase ])
        #immediate_processingQueue.to_csv("impq.csv", sep=',', encoding='utf-8')
        


        #print('In Phase 2',len(immediate_processingQueue))
        #immediate_processingQueue=immediate_processingQueue.reset_index(drop=True)
        combined_list_here=([]+list(cachedStopWords)+chat_word_list+day_list+month_list+article_list+prep_list)
        combined_list_filtered=list(filter(lambda word: word not in (prep_list+article_list+month_list+self.phase2stopwordList), combined_list_here))
        #--------------------------------------PHASE II---------------------------------------------------
        for index, row in tweetBaseInput.iterrows():

            #phase 1 candidates for one sentence
            phase1_holder=[]

            tweetText=str(row['TweetSentence'])
            tweetWordList = row['tweetwordList']
            sentID=str(row['sentID'])
            tweetID=str(row['tweetID'])
            phase1Candidates=str(row['phase1CandidatesWPositions'])
            batch=int(row['entry_batch'])
            contextual_embeddings_dict= {ind: embedding for ind, embedding in enumerate(row['contextual_embeddings'])}
            
            # print('====',tweetID,sentID)
            # print('tweetWordList:',tweetWordList)
            
            non_discriminative_flag=False
            phase1CandidatesList=[]

            # print('phase1Candidates:',phase1Candidates)

            if (phase1Candidates !='nan'):
                phase1Raw=phase1Candidates.split("||")
                phase1Raw = list(filter(None, phase1Raw))


                for entities_with_loc in phase1Raw:
                    entity_to_store=entities_with_loc.split("::")[0]
                    #print(entity_to_store)
                    position=entities_with_loc.split("::")[1]
                    #print(position)
                    phase1_holder.append((entity_to_store,position))
                    phase1_holder.clear()
                    phase1CandidatesList.append(entity_to_store.lower())

                phase1_holder_holder.append(copy.deepcopy(phase1_holder))
                

            else:
                non_discriminative_flag=True
                phase1_holder_holder.append([])

            
            # tweetWordList=self.getWords(tweetText)
            # tweetWordList= [(token,idx) for idx,token in enumerate(tweetWordList)]
            
            tweetWordList_stopWords=list(filter (lambda word: ((((word[0].strip()).strip(string.punctuation)).lower() in combined_list_filtered)|(word[0].strip() in string.punctuation)|(word[0].startswith('@'))|(word[0].startswith('#'))), tweetWordList))

            # phase 2 candidate tuples without stopwords for a sentence
            c=[(y[0],str(y[1]),tweetID,sentID,'ne',batch,time) for y  in tweetWordList if y not in tweetWordList_stopWords ]
            #c=[(y[0],str(y[1])) for y  in tweetWordList if y not in tweetWordList_stopWords ]

            
            sequences=[]
            for k, g in groupby(enumerate(c), lambda element: element[0]-int(element[1][1])):
                sequences.append(list(map(itemgetter(1), g)))

            # print('phaseII candidates:')
            ne_candidate_list=[]
            for sequence in sequences:
                seq_candidate_list=self.get_Candidates(sequence, CTrie,False)
                if(seq_candidate_list):
                    for candidate_tuple in seq_candidate_list:
                        # print(candidate_tuple)

                        # extract candidate token embeddings
                        candidate_token_embeddings = torch.stack([contextual_embeddings_dict[int(position)] for position in candidate_tuple[1]])
                        # print('candidate with token_embeddings:',candidate_tuple[0],candidate_token_embeddings.shape,len(candidate_tuple[1]))

                        # !!necessary because this function during training receives [1,n,100] tensors that it squeezes; so might screw up 1-token sentences
                        candidate_embedding = self.entity_phrase_embedder.getEmbedding(candidate_token_embeddings.unsqueeze(0))
                        # print('candidate_embedding:',candidate_embedding.shape)

                        #inserts into CandidateBase: Syntax and Context Feature Setting
                        if not ((float(batch)<self.counter)&(candidate_tuple[-1]<self.counter)):
                            self.update_Candidatedict(candidate_tuple,non_discriminative_flag,candidate_embedding.tolist())

                    ne_candidate_list.extend(seq_candidate_list)

            phase2_candidates=[self.normalize(e[0]) for e in ne_candidate_list]
            phase2_candidates_unnormalized=[e[0] for e in ne_candidate_list]

            phase2_candidates_holder.append(phase2_candidates)
            phase2_unnormalized_candidates_holder.append(phase2_candidates_unnormalized)

            dict1 = {'entry_batch':batch, 'tweetID':tweetID, 'sentID':sentID, 'TweetSentence':tweetText, 'phase1Candidates':phase1CandidatesList,'2nd Iteration Candidates':phase2_candidates,'2nd Iteration Candidates Unnormalized':phase2_candidates_unnormalized}

            df_holder.append(dict1)
            
            #-------------------------------------------------------------------END of 1st iteration: RESCAN+CANDIDATE_UPDATION-----------------------------------------------------------

        # #===============comment out if not gathering records for classifier training===============

        # candidate_records=pd.read_csv('data/candidate_train_records_large_300d.csv',sep =',',keep_default_na=False)
        # candidate_records = candidate_records[['candidate','class']]
        # candidate_list = candidate_records.candidate.values.tolist()

        # candidateBase_dict_filtered={}
        # for candidate in self.CandidateBase_dict:
        #     if(candidate in candidate_list):
        #         candidateBase_dict_filtered[candidate] = self.CandidateBase_dict[candidate]

        # # candidateBase_dict_filtered = self.CandidateBase_dict

        # candidate_featureBase_DF_filtered=pd.DataFrame.from_dict(candidateBase_dict_filtered, orient='index')
        # candidate_featureBase_DF_filtered.columns=self.candidateBaseHeaders[1:]
        # candidate_featureBase_DF_filtered.index.name=self.candidateBaseHeaders[0]
        # candidate_featureBase_DF_filtered = candidate_featureBase_DF_filtered.reset_index(drop=False)
        # # candidate_featureBase_DF_filtered = candidate_featureBase_DF_filtered[candidate_featureBase_DF_filtered['cumulative'] >= 5.0]

        # print('before:', len(self.CandidateBase_dict),'after:',len(candidateBase_dict_filtered))
        
        # candidate_records.set_index('candidate', inplace=True)
        # candidate_featureBase_DF_filtered['class'] = candidate_featureBase_DF_filtered.apply(lambda row:candidate_records.loc[row.candidate]['class'] , axis = 1)

        # # candidate_featureBase_DF_filtered['class'] = 0

        # candidate_featureBase_DF_filtered.to_csv("data/candidate_train_records_large_100d.csv", sep=',', encoding='utf-8')
        # return #comment out if not collecting records for classifier training
        # #===============comment out if not gathering records for classifier training===============

        candidate_featureBase_DF = pd.DataFrame.from_dict(self.CandidateBase_dict, orient='index')
        candidate_featureBase_DF.columns = self.candidateBaseHeaders[1:]
        candidate_featureBase_DF.index.name = self.candidateBaseHeaders[0]
        candidate_featureBase_DF = candidate_featureBase_DF.reset_index(drop=False)

        return candidate_featureBase_DF,df_holder,phase2_candidates_holder,phase2_unnormalized_candidates_holder


        ## self.aggregator_incomplete_tweets= self.aggregator_incomplete_tweets.append(self.incomplete_tweets)
        ## self.just_converted_tweets=self.just_converted_tweets.append(just_converted_tweets_for_current_batch)


    def finish(self):
        return self.accuracy_vals

    def finish_other_systems(self):
        stanford_f1=[]
        stanford_precision=[]
        stanford_recall=[]
        print("*****************************************STANFORD***********************")
        for i in self.accuracy_vals_stanford:
            stanford_f1.append(i[0])
            stanford_precision.append(i[1])
            stanford_recall.append(i[2])
            # print(i)
        print('stanford_f1:', stanford_f1)
        print('stanford_precision:', stanford_precision)
        print('stanford_recall:', stanford_recall)

        print(sum(stanford_f1)/len(stanford_f1))
        print(sum(stanford_precision)/len(stanford_precision))
        print(sum(stanford_recall)/len(stanford_recall))

        print("*****************************************Opencalai***********************")
        opencalai_f1=[]
        opencalai_precision=[]
        opencalai_recall=[]
        for i in self.accuracy_vals_opencalai:
            opencalai_f1.append(i[0])
            opencalai_precision.append(i[1])
            opencalai_recall.append(i[2])
        print('opencalai_f1:', opencalai_f1)
        print('opencalai_precision:', opencalai_precision)
        print('opencalai_recall:', opencalai_recall)

        print(sum(opencalai_f1)/len(opencalai_f1))
        print(sum(opencalai_precision)/len(opencalai_precision))
        print(sum(opencalai_recall)/len(opencalai_recall))
        print("*****************************************Ritter***********************")
        ritter_f1=[]
        ritter_precision=[]
        ritter_recall=[]
        for i in self.accuracy_vals_ritter:
            ritter_f1.append(i[0])
            ritter_precision.append(i[1])
            ritter_recall.append(i[2])
        print('ritter_f1:', ritter_f1)
        print('ritter_precision:', ritter_precision)
        print('ritter_recall:', ritter_recall)

        print(sum(ritter_f1)/len(ritter_f1))
        print(sum(ritter_precision)/len(ritter_precision))
        print(sum(ritter_recall)/len(ritter_recall))
        print("*****************************************Neuroner***********************")
        neuroner_f1=[]
        neuroner_precision=[]
        neuroner_recall=[]
        for i in self.accuracy_vals_neuroner:
            neuroner_f1.append(i[0])
            neuroner_precision.append(i[1])
            neuroner_recall.append(i[2])
        print('neuroner_f1:', neuroner_f1)
        print('neuroner_precision:', neuroner_precision)
        print('neuroner_recall:',neuroner_recall)

        print(sum(neuroner_f1)/len(neuroner_f1))
        print(sum(neuroner_precision)/len(neuroner_precision))
        print(sum(neuroner_recall)/len(neuroner_recall))

        return (self.accuracy_vals_stanford,self.accuracy_vals_opencalai,self.accuracy_vals_ritter,self.accuracy_vals_neuroner)

## **Running the Engine**

In [ ]:

# tweets_unpartitoned=pd.read_csv('data/venezuela/venezuela.csv',sep =',',keep_default_na=False)

tweets_unpartitoned=pd.read_csv("data/covid/covid_2K.csv",sep =',',keep_default_na=False)

# tweets_unpartitoned=pd.read_csv('data/tweets_3K/tweets_3k_annotated.csv',sep =',',keep_default_na=False)

# tweets_unpartitoned=pd.read_csv('data/roevwade/roevwade.csv',sep =',',keep_default_na=False)

# tweets_unpartitoned=pd.read_csv('data/billdeblasio/billdeblasio.csv',sep =',',keep_default_na=False)

# tweets_unpartitoned=pd.read_csv('data/pikapika/pikapika.csv',sep =',',keep_default_na=False)

# tweets_unpartitoned=pd.read_csv('data/ripcity/ripcity.csv',sep =',',keep_default_na=False)

# tweets_unpartitoned=pd.read_csv('data/billnye/billnye.csv',sep =',',keep_default_na=False)

# tweets_unpartitoned=pd.read_csv('data/wnut-test/wnut17test.csv',sep =',',keep_default_na=False)

# tweets_unpartitoned=pd.read_csv('data/broad_twitter_corpus/broad_twitter_corpus.csv',sep =',',keep_default_na=False)

#to train the Entity Classifiers
# tweets_unpartitoned=pd.read_csv('data/deduplicated/deduplicated_test.csv',sep =';',keep_default_na=False)

In [ ]:
print('Tweets are in memory...')

length=len(tweets_unpartitoned)
batch_size=length
print(length, batch_size)
val=math.ceil(length/batch_size)-1

Tweets are in memory...
1998 1998


In [ ]:
# from executor import *
# sentence_df_dict_gaguilar, tweet_to_sentences_w_annotation = execute('emerging.test.conll')
# # print(token_feature_tuple_list[:3])

from executor import *

# trained_word2index, trained_index2postag, trained_gaze2index, trained_gaze_inputs, trained_char_inputs, trained_word_inputs, trained_mtl_network, trained_fextractor = prepare_for_training()
# sentence_df_dict_gaguilar, tweet_to_sentences_w_annotation = execute('emerging.test.conll', trained_word2index, trained_index2postag, trained_gaze2index, trained_gaze_inputs, trained_char_inputs, trained_word_inputs, trained_mtl_network, trained_fextractor)

tweets_test, labels_test, postag_test, trained_word2index, trained_index2postag, trained_gaze2index, trained_gaze_inputs, trained_char_inputs, trained_word_inputs, trained_mtl_network, trained_fextractor = prepare_for_training('emerging.test.conll')
# tweets_test, labels_test, postag_test, trained_word2index, trained_index2postag, trained_gaze2index, trained_gaze_inputs, trained_char_inputs, trained_word_inputs, trained_mtl_network, trained_fextractor = prepare_for_training('btc.emerging.test.conll')
# tweets_test, labels_test, postag_test, trained_word2index, trained_index2postag, trained_gaze2index, trained_gaze_inputs, trained_char_inputs, trained_word_inputs, trained_mtl_network, trained_fextractor = prepare_for_training('deduplicated.emerging.test.conll')

sentence_df_dict_gaguilar, tweet_to_sentences_w_annotation = execute(tweets_test, labels_test, postag_test, trained_word2index, trained_index2postag, trained_gaze2index, trained_gaze_inputs, trained_char_inputs, trained_word_inputs, trained_mtl_network, trained_fextractor)

sentence_df_dict format: tweetID, sentID, sentence, list of extracted mentions, token wise 100-d features from DNN;;
key-> serialized sentence #

In [ ]:
tweet_batch = tweets_unpartitoned
print(len(tweet_batch))

total_time=0
g=0
reintroduction_threshold_dummy=0
max_batch_value=112
z_score=1

In [ ]:
#Running Phase I
local_NER_Module= LocalNERModule(sentence_tokenizer, sentence_df_dict_gaguilar, tweet_to_sentences_w_annotation, device)


In [ ]:
return_tuple = local_NER_Module.extract(g)

tweet_base=return_tuple[0]
contextual_embeddings=return_tuple[1]
candidate_base=return_tuple[2]
elapsedTime= return_tuple[4] - return_tuple[3]
phase2stopwordList=return_tuple[5]
# print('len of tweet_base = '  len(tweet_base))
tweet_to_sentences_w_annotation=return_tuple[6]
total_time+=elapsedTime
print(elapsedTime,total_time)

# df_out_holder_Phase1.append(tweet_base)

print ('Produced', g)
print("**********************************************************")


In [ ]:
# global_NER_Module = GlobalNERModule(entityPhraseEmbedder, device,'deduplicated_test')

# global_NER_Module = GlobalNERModule(entityPhraseEmbedder, device,'venezuela')

global_NER_Module = GlobalNERModule(entityPhraseEmbedder, device,'covid')

# global_NER_Module = GlobalNERModule(entityPhraseEmbedder, device,'tweets_3K')

# global_NER_Module = GlobalNERModule(entityPhraseEmbedder, device,'roevwade')

# global_NER_Module = GlobalNERModule(entityPhraseEmbedder, device,'billdeblasio')

# global_NER_Module = GlobalNERModule(entityPhraseEmbedder, device,'pikapika')

# global_NER_Module = GlobalNERModule(entityPhraseEmbedder, device,'billnye')

# global_NER_Module = GlobalNERModule(entityPhraseEmbedder, device,'ripcity')

# global_NER_Module = GlobalNERModule(entityPhraseEmbedder, device,'wnut17')

# global_NER_Module = GlobalNERModule(entityPhraseEmbedder, device,'btc')

In [ ]:
time_in = time.time()
candidate_base_post_Phase2, complete_tweet_dataframe,time_out = global_NER_Module.executor(max_batch_value,tweet_base,candidate_base,phase2stopwordList,z_score,reintroduction_threshold_dummy,tweet_batch)
# time_out = time.time()

print('time_taken', (time_out-time_in))

## **Extracting Embeddings to train the Phrase Embedder on STS Dataset-- (NOT REQD TO RUN) unless training the phrase Embedder**

In [ ]:
class PhraseEmbeddingDataset(torch.utils.data.Dataset):

    def __init__(self, input, output):

        # print(data[0])
        # self.data = np.asarray(data)
        # self.output = np.asarray(output)

        self.input = input
        self.output = output

        print(type(self.input),type(self.output))

    def __len__(self):
        assert len(self.input) == len(self.output)
        return len(self.input)

    def __getitem__(self, idx):
        X = self.input[idx]
        y = self.output[idx]
        return X,y

In [ ]:
def compile_token_embeddings(lst):
    token_embeddings=[]
    for tup in lst:
        token = tup[0]
        # tup[1] is feature dict, keys 0 to 99
        embedding_dict = tup[1]
        embedding_arr = np.array([embedding_dict['feat'+str(i)] for i in range(0,100)])
        token_embeddings.append(torch.from_numpy(embedding_arr).float())
    return token_embeddings

def quick_extract_sentence_token_embeddings(gaguilar_sentence_outputs):
    entity_embeddings=[]
    for key, value in gaguilar_sentence_outputs.items():
        token_embedding_list= list(value[4])
        entity_aware_embeddings = compile_token_embeddings(token_embedding_list)
        entity_embeddings.append(torch.stack(entity_aware_embeddings))
    return entity_embeddings

In [ ]:
def get_sts_data_scores(filename):
    stsDataDictList=[]
    stsData_columns =['sentence1', 'sentence2', 'score']
    f=open("data/stsbenchmark/"+filename+".csv",'r')
    file_text= f.read()
    lines=file_text.split('\n')
    for line in lines:
        if(line):
            fields=line.split('\t')
    #         print(len(fields))
            dataDict={'sentence1':fields[5],'sentence2':fields[6],'score':float(fields[4])/5.0}
    #         print(dataDict)
            stsDataDictList.append(dataDict)
    stsData=pd.DataFrame(stsDataDictList)
    return stsData['score'].tolist()

stsTrainDataScores = get_sts_data_scores('sts-train')
print(len(stsTrainDataScores))
# print(stsTrainData.columns)

stsDevDataScores = get_sts_data_scores('sts-dev')
print(len(stsDevDataScores))
# print(stsDevData.columns)

In [ ]:
from executor import *
trained_word2index, trained_index2postag, trained_gaze2index, trained_gaze_inputs, trained_char_inputs, trained_word_inputs, trained_mtl_network, trained_fextractor = prepare_for_training()


In [ ]:
# from executor import *

#Training Data
#For Source Sentences:
# training_source_sentence_df_dict_gaguilar, training_source_tweet_to_sentences_w_annotation = execute('sts.sentence1.train.conll')
training_source_sentence_df_dict_gaguilar, training_source_tweet_to_sentences_w_annotation = execute('sts.sentence1.train.conll', trained_word2index, trained_index2postag, trained_gaze2index, trained_gaze_inputs, trained_char_inputs, trained_word_inputs, trained_mtl_network, trained_fextractor)
source_sentence_embeddings_train = quick_extract_sentence_token_embeddings(training_source_sentence_df_dict_gaguilar)

In [ ]:
#Training Data
#For Target Sentences:
# training_target_sentence_df_dict_gaguilar, training_target_tweet_to_sentences_w_annotation = execute('sts.sentence2.train.conll')
training_target_sentence_df_dict_gaguilar, training_target_tweet_to_sentences_w_annotation = execute('sts.sentence2.train.conll', trained_word2index, trained_index2postag, trained_gaze2index, trained_gaze_inputs, trained_char_inputs, trained_word_inputs, trained_mtl_network, trained_fextractor)
target_sentence_embeddings_train = quick_extract_sentence_token_embeddings(training_target_sentence_df_dict_gaguilar)
assert len(source_sentence_embeddings_train)==len(target_sentence_embeddings_train)

In [ ]:
#Validation Data
#For Source Sentences:
# dev_source_sentence_df_dict_gaguilar, dev_source_tweet_to_sentences_w_annotation = execute('sts.sentence1.dev.conll')
dev_source_sentence_df_dict_gaguilar, dev_source_tweet_to_sentences_w_annotation = execute('sts.sentence1.dev.conll', trained_word2index, trained_index2postag, trained_gaze2index, trained_gaze_inputs, trained_char_inputs, trained_word_inputs, trained_mtl_network, trained_fextractor)
source_sentence_embeddings_dev = quick_extract_sentence_token_embeddings(dev_source_sentence_df_dict_gaguilar)

In [ ]:
#Validation Data
#For Target Sentences:
# dev_target_sentence_df_dict_gaguilar, dev_target_tweet_to_sentences_w_annotation = execute('sts.sentence2.dev.conll')
dev_target_sentence_df_dict_gaguilar, dev_target_tweet_to_sentences_w_annotation = execute('sts.sentence2.dev.conll', trained_word2index, trained_index2postag, trained_gaze2index, trained_gaze_inputs, trained_char_inputs, trained_word_inputs, trained_mtl_network, trained_fextractor)
target_sentence_embeddings_dev = quick_extract_sentence_token_embeddings(dev_target_sentence_df_dict_gaguilar)
assert len(source_sentence_embeddings_dev)==len(target_sentence_embeddings_dev)

In [ ]:
#Printing some shapes
print(len(source_sentence_embeddings_train), len(target_sentence_embeddings_train))
print(source_sentence_embeddings_train[0].shape)
embeddingSize=list(source_sentence_embeddings_train[0][0].shape)[0]

In [ ]:
# Datasets and DataLoaders
training_set = PhraseEmbeddingDataset(list(map(list, zip(source_sentence_embeddings_train, target_sentence_embeddings_train))),stsTrainDataScores)
training_generator = torch.utils.data.DataLoader(training_set, shuffle=True)

validation_set = PhraseEmbeddingDataset(list(map(list, zip(source_sentence_embeddings_dev, target_sentence_embeddings_dev))),stsDevDataScores)
validation_generator = torch.utils.data.DataLoader(validation_set, shuffle=True)

In [ ]:
def save_ckp(state, is_best, checkpoint_dir):
    # f_path = checkpoint_dir + '/checkpoint.pt' 
    f_path = checkpoint_dir + '/checkpoint_model100.pt' #_model100
    torch.save(state, f_path)
    # if is_best:
    #     best_fpath = best_model_dir +'/best_model.pt'
    #     shutil.copyfile(f_path, best_fpath)

In [ ]:
# Initialize network
phraseEmbeddingModel = PhraseEmbedding(embeddingSize, output_embedding_size, device).to(device)

#Loss and Optimizer
criterion = nn.MSELoss(reduction='mean' )
optimizer = optim.Adam(phraseEmbeddingModel.parameters(), lr=learning_rate, weight_decay=0.0001)

In [ ]:
checkpoint_dir='entityEmbedding/model_checkpoints'

#Hyperparameters
learning_rate = 0.0001
num_epochs = 200
patience = 5

# Train Network
history_training= []
history_validation = []
best_loss = np.float('inf')

for epoch in range(num_epochs):
    training_batch_loss=[]
    for batch_idx, (data, target) in enumerate(training_generator):
        target = torch.tensor(float(target)).to(device=device)
        out = phraseEmbeddingModel(data)
        # print(out.item())
        # if(not math.isnan(out.item())):
            # print(data)
        loss = criterion(out, target)
        # print(loss.item())
        training_batch_loss.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(phraseEmbeddingModel.parameters(), 1.0)
        # gradient descent or adam step
        optimizer.step()
    combined_training_loss = np.mean(training_batch_loss)
    print('combined epoch training loss:', combined_training_loss)
    history_training.append(combined_training_loss)

    #Validation: DO NOT BACKPROPAGATE HERE
    validation_batch_loss = []
    print_only_one=True
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(validation_generator):
            target = torch.tensor(float(target)).to(device=device)
            if(print_only_one):
                # print(len(data[0]),len(data[1]))
                # print(type(data))
                print_only_one=False
            out = phraseEmbeddingModel(data)
            loss = criterion(out, target)
            validation_batch_loss.append(loss.item())
            # print(validation_batch_loss)
    combined_validation_loss= np.mean(validation_batch_loss)
    history_validation.append(combined_validation_loss)
    # if(((epoch+1)%10==0)|(epoch == (num_epochs-1))):
    print('\nEpoch',str(epoch+1),' Validation Loss:',combined_validation_loss)
    
    if(combined_validation_loss<best_loss):
        best_loss = combined_validation_loss
        print('saving this checkpoint')
        checkpoint = {
            'epoch': epoch + 1,
            'state_dict': phraseEmbeddingModel.state_dict(),
            'optimizer': optimizer.state_dict()
        }
        save_ckp(checkpoint, True, checkpoint_dir)
        no_improvement_counter=0
        print('=====================================================================\n')
    else:
        no_improvement_counter+=1
        if(no_improvement_counter>patience):
            break


In [ ]:
checkpoint_path = 'entityEmbedding/model_checkpoints/checkpoint_model100.pt' #300
# model_dir = 'best-model'
# save_ckp(checkpoint, True, checkpoint_dir, model_dir)

# load the last checkpoint with the best model
entityPhraseEmbedder, optimizer, start_epoch = load_ckp(checkpoint_path, phraseEmbeddingModel, optimizer)
print('Loading model from epoch:', start_epoch)

## **Regular EMD Runs (NO NEED TO RUN)**

In [ ]:
# !python3 main.py btc.emerging.test.conll
# !python3 main.py emerging.test.conll